# ASSIGNMENT 3
# Submission Deadline: 20/03/2024 at 10 AM
# Submission Link: https://forms.gle/b8s6xYHUYqTJtSNeA

# Table of Contents

1. [Provide Information](#Provide-Information)
2. [Instructions](#Instructions)
3. [Environment](#Environment)
4. [Hyperparameters](#Hyperparameters)
5. [Helper Functions](#helper)
6. [Deep Value Based RL Agents](#deep-value-based)
7. [Deep Policy Based RL Agents](#deep-policy-based)
8. [Experiments to Run](#experiments)

# Provide Information
<a id="Provide-Information"></a>

Name: **TA**

Roll No.: **-**

IITK EMail: **-**

# Instructions
<a id="Instructions"></a>


**Read all the instructions below carefully before you start working on the assignment.**
- The purpose of this course is that you learn RL and the best way to do that is by implementation and experimentation.
- The assignment requires your to implement some algorithms and you are required report your findings after experimenting with those algorithms.
- **You are required to submit ZIP file containing a Jupyter notebook (.ipynb), and an image folder. The notebook would include the code, graphs/plots of the experiments you run and your findings/observations. Image folder is the folder having plots, images, etc.**
- In case you use any maths in your explanations, render it using latex in the Jupyter notebook.
- You are expected to implement algorithms on your own and not copy it from other sources/class mates. Of course, you can refer to lecture slides.
- If you use any reference or material (including code), please cite the source, else it will be considered plagiarism. But referring to other sources that directly solve the problems given in the assignment is not allowed. There is a limit to which you can refer to outside material.
- This is an individual assignment.
- In case your solution is found to have an overlap with solution by someone else (including external sources), all the parties involved will get zero in this and all future assignments plus further more penalties in the overall grade. We will check not just for lexical but also semantic overlap. Same applies for the code as well. Even an iota of cheating would NOT be tolerated. If you cheat one line or cheat one page the penalty would be same.
- Be a smart agent, think long term, if you cheat we will discover it somehow, the price you would be paying is not worth it.
- In case you are struggling with the assignment, seek help from TAs. Cheating is not an option! I respect honesty and would be lenient if you are not able to solve some questions due to difficulty in understanding. Remember we are there to help you out, seek help if something is difficult to understand.
- The deadline for the submission is given above. Submit at least 30 minutes before the deadline, lot can happen at the last moment, your internet can fail, there can be a power failure, you can be abducted by aliens, etc.
- You have to submit your assignment via the Google Form (link above)
- The form would close after the deadline and we will not accept any solution. No reason what-so-ever would be accepted for not being able to submit before the deadline.
- Since the assignment involves experimentation, reporting your results and observations, there is a lot of scope for creativity and innovation and presenting new perspectives. Such efforts would be highly appreciated and accordingly well rewarded. Be an exploratory agent!
- Your code should be very well documented, there are marks for that.
- In your plots, have a clear legend and clear lines, etc. Of course you would generating the plots in your code but you must also put these plots in your notebook. Generate high resolution pdf/svg version of the plots so that it doesn't pixilate on zooming.
- For all experiments, report about the seed used in the code documentation, write about the seed used.
- In your notebook write about all things that are not obvious from the code e.g., if you have made any assumptions, references/sources, running time, etc.
-  **DO NOT Forget to write name, roll no and email details above**
- **In addition to checking your code, we will be conducting one-on-one viva for the evaluation. So please make sure that you do not cheat!**
- **Use of LLMs based tools or AI-based code tools is strictly prohibited! Use of ChatGPT, VS Code, Gemini, CO-Pilot, etc. is not allowed. NOTE VS code is also not allowed. Even in Colab disable the AI assistant. If you use it, we will know it very easily. Use of any of the tools would be counted as cheating and would be given a ZERO, with no questions asked.**
- For each of the sub-part in the question create a new cell below the question and put your answer in there. This includes the plots as well

# OpenAI Gym Environments
<a id="Environment"></a>

In [ ]:
!pip install torch

In [ ]:
!pip install gymnasium

In [ ]:
import numpy
print(numpy.__version__)

In [ ]:
# all imports go in here
import gymnasium as gym
import numpy as np
import random
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from itertools import cycle, count
import time

In this assignment we will be exploring Deep RL algorithms and for this we will be using environmentd provided by OpenAI Gym. In particualr we will be exploring "CartPole-v0" and "MountainCar-v0" environments (https://gymnasium.farama.org/environments/classic_control/ ). The code to instantiate the environments are given in the cells below. Run these cells and play with the environments to learn more details about the environments.

In [ ]:
# Create CartPole environment
#https://gymnasium.farama.org/environments/classic_control/cart_pole/

env = gym.make('CartPole-v0')
#env.seed(34)
s = env.reset(seed=344)
print("Observation Space = ")
print(env.observation_space)
print("Action Space = ")
print(env.action_space)
done = False
for episode in range(20):
    print("In episode {}".format(episode))
    for i in range(100):
        env.render()
        print(s)
        a = env.action_space.sample()
        s, r, done, truncated,_ = env.step(a)
        if done:
            print("Finished after {} timestep".format(i+1))
env.close()

In [ ]:
# Create MountainCar environment:
# https://gymnasium.farama.org/environments/classic_control/mountain_car/

env = gym.make('MountainCar-v0', render_mode="rgb_array")
#env.seed(45)
s = env.reset(seed=455)
print("Observation Space = ")
print(env.observation_space)
print("Action Space = ")
print(env.action_space)
done = False
for episode in range(20):
    print("In episode {}".format(episode))
    for i in range(100):
        env.render()
        print(s)
        a = env.action_space.sample()
        s, r, done, truncated, _ = env.step(a)
        if done:
            print("Finished after {} timestep".format(i+1))
env.close()

# Hyperparameters
<a id="Hyperparameters"></a>

All your hyperparameters should be stated here. We will change their value here and your code should work  accordingly.

In [ ]:
# mention the values of all the hyperparameters (you can add more hyper-paramters as well) to be used in the entire notebook, put the values that gave the best
# performance and were finally used for the agent

gamma =  #gamma
epsilon = #epsolon greedy strategy
temp = #softmax strategy
delta = #huber loss
tau = #D3QN
alpha = #D3QN-PER
beta = #D3QN-PER
beta_rate = #D3QN-PER
MAX_TRAIN_EPISODES = 500
MAX_EVAL_EPISODES = 1

# Helper Functions
<a id="helper"></a>

Write all the helper functions that will be used for value-based and policy based algorithms below. In case you want to add more helper functions, please feel free to add.

In [ ]:
def selectGreedyAction(net, state):
    #this function gets q-values via the network and selects greedy action from q-values and returns it

    #Your code goes in here
    with torch.no_grad():
        # Assuming state is already a PyTorch tensor and appropriately preprocessed
        q_values = net(state).cpu().detach().numpy().squeeze()
        # Select the action with the highest Q-value
        return np.argmax(q_values)



In [ ]:
def selectEpsilonGreedyAction(net,  state, epsilon):
    #this function gets q-values via the network and selects an action from q-values using epsilon greedy strategy
    #and returns it
    #note this function can be used for decaying epsilon greedy strategy,
    #you would need to create a wrapper function that will handle decaying epsilon
    #you can create this wrapper in this helper function section
    #for the agents you would be implementing it would be nice to play with decaying parameter to get optimal results

    #Your code goes in here
    exploration = False
    with torch.no_grad():
        # Assuming state is already a PyTorch tensor and appropriately preprocessed
        q_values = net(state).cpu().detach().numpy().squeeze()

    if np.random.rand() > epsilon:
        action = np.argmax(q_values)
    else:
        action = np.random.randint(len(q_values))
        exploration = True


    return action



In [ ]:
def selectEpsilonExpGreedyAction(model, state, init=False, init_epsilon=1.0, min_epsilon=0.3, decay_steps=20000):
    if init or not hasattr(selectEpsilonExpGreedyAction, 'initialized'):
        # Initialization
        selectEpsilonExpGreedyAction.epsilon = init_epsilon
        selectEpsilonExpGreedyAction.init_epsilon = init_epsilon
        selectEpsilonExpGreedyAction.decay_steps = decay_steps
        selectEpsilonExpGreedyAction.min_epsilon = min_epsilon
        selectEpsilonExpGreedyAction.epsilons = 0.01 / np.logspace(-2, 0, decay_steps, endpoint=False) - 0.01
        selectEpsilonExpGreedyAction.epsilons = selectEpsilonExpGreedyAction.epsilons * (init_epsilon - min_epsilon) + min_epsilon
        selectEpsilonExpGreedyAction.t = 0
        selectEpsilonExpGreedyAction.initialized = True
        return -1

    # Epsilon update
    selectEpsilonExpGreedyAction.epsilon = selectEpsilonExpGreedyAction.min_epsilon if selectEpsilonExpGreedyAction.t >= selectEpsilonExpGreedyAction.decay_steps else selectEpsilonExpGreedyAction.epsilons[selectEpsilonExpGreedyAction.t]
    selectEpsilonExpGreedyAction.t += 1

    # Action selection
    with torch.no_grad():
        q_values = model(state).detach().cpu().data.numpy().squeeze()

    if np.random.rand() > selectEpsilonExpGreedyAction.epsilon:
        action = np.argmax(q_values)
    else:
        action = np.random.randint(len(q_values))

    return action


In [ ]:
def selectSoftMaxAction(net, state, temp):
    #this function gets q-values via the network and selects an action from q-values using softmax strategy
    #and returns it
    #note this function can be used for decaying temperature softmax strategy,
    #you would need to create a wrapper function that will handle decaying temperature
    #you can create this wrapper in this helper function section
    #for the agents you would be implementing it would be nice to play with decaying parameter to get optimal results

    #Your code goes in here
    with torch.no_grad():
        # Assuming state is already a PyTorch tensor and appropriately preprocessed
        q_values = net(state).cpu().detach().numpy().squeeze()
        # Compute softmax probabilities with temperature temp
        # Adjust q_values by temp and avoid overflow with max subtraction
        q_values_adj = q_values / temp - np.max(q_values / temp)
        probabilities = np.exp(q_values_adj) / np.sum(np.exp(q_values_adj))
        # Select an action based on the probabilities
        action = np.random.choice(len(q_values), p=probabilities)
        return action

    return softAction

In [ ]:
#Value Network

def createValueNetwork(inDim, outDim, hDim=[32, 32], activation=F.relu):
    #this creates a Feed Forward Neural Network class and instantiates it and returns the class
    #the class should be derived from torch nn.Module and it should have init and forward method at the very least
    #the forward function should return q-value for each possible action
    class MyValueNetwork(nn.Module):
        def __init__(self, input_dim, output_dim, hidden_dims, activation_fc):
            super(MyValueNetwork, self).__init__()
            self.activation_fc = activation_fc

            # Device management
            self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

            # Network layers
            self.input_layer = nn.Linear(input_dim, hidden_dims[0])
            self.hidden_layers = nn.ModuleList()
            for i in range(len(hidden_dims) - 1):
                self.hidden_layers.append(nn.Linear(hidden_dims[i], hidden_dims[i + 1]))
            self.output_layer = nn.Linear(hidden_dims[-1], output_dim)

            self.to(self.device)

        def _format(self, state):
            if not isinstance(state, torch.Tensor):
                state = torch.tensor(state, device=self.device, dtype=torch.float32)
            if state.dim() == 1:
                state = state.unsqueeze(0)
            return state

        def forward(self, state):
            x = self._format(state)
            x = self.activation_fc(self.input_layer(x))
            for hidden_layer in self.hidden_layers:
                x = self.activation_fc(hidden_layer(x))
            x = self.output_layer(x)
            return x

        def load(self, experiences):
            states, actions, rewards, new_states, is_terminals = experiences
            states = torch.from_numpy(states).float().to(self.device)
            actions = torch.from_numpy(actions).long().to(self.device)
            new_states = torch.from_numpy(new_states).float().to(self.device)
            rewards = torch.from_numpy(rewards).float().to(self.device)
            is_terminals = torch.from_numpy(is_terminals).float().to(self.device)
            return states, actions, rewards, new_states, is_terminals

    return MyValueNetwork(inDim, outDim, hDim, activation)





In [ ]:
#Dueling Network
def createDuelingNetwork(input_dim, output_dim, hidden_dims=[32, 32], activation_fn=F.relu):
    class DDQNetwork(nn.Module):
        def __init__(self, in_features, out_features, hidden_layers=(32, 32), activation=activation_fn):
            super(DDQNetwork, self).__init__()
            self.activate = activation

            # Device management
            self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
            self.initial_layer = nn.Linear(in_features, hidden_layers[0])
            self.hidden_layers = nn.ModuleList()
            for i in range(len(hidden_layers) - 1):
                layer = nn.Linear(hidden_layers[i], hidden_layers[i + 1])
                self.hidden_layers.append(layer)
            self.value_stream = nn.Linear(hidden_layers[-1], 1)
            self.advantage_stream = nn.Linear(hidden_layers[-1], out_features)

            self.computing_device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
            self.to(self.computing_device)

        def _format(self, state):
            if not isinstance(state, torch.Tensor):
                state = torch.tensor(state, dtype=torch.float32, device=self.computing_device)
                state = state.unsqueeze(0)
            return state

        def forward(self, state):
            state = self._format(state)
            state = self.activate(self.initial_layer(state))
            for layer in self.hidden_layers:
                state = self.activate(layer(state))
            value = self.value_stream(state)
            advantage = self.advantage_stream(state)
            q_value = value + advantage - advantage.mean(dim=1, keepdim=True).expand_as(advantage)
            return q_value

        def numpy_float_to_device(self, data):
            return torch.from_numpy(data).float().to(self.computing_device)

        def load(self, experiences):
            states, actions, new_states, rewards, is_terminals = experiences
            states = torch.from_numpy(states).float().to(self.device)
            actions = torch.from_numpy(actions).long().to(self.device)
            new_states = torch.from_numpy(new_states).float().to(self.device)
            rewards = torch.from_numpy(rewards).float().to(self.device)
            is_terminals = torch.from_numpy(is_terminals).float().to(self.device)
            return states, actions, new_states, rewards, is_terminals
            
    return DDQNetwork(input_dim, output_dim, hidden_dims, activation_fn)

In [ ]:
#Policy Network
def createPolicyNetworkREINFORCE(inDim, outDim, hDim = [32,32], activation = F.relu):
    #this creates a Feed Forward Neural Network class and instantiates it and returns the class
    #the class should be derived from torch nn.Module and it should have init and forward method at the very least
    #the forward function should return action logit vector
    #Your code goes in here

    class policyNetwork(nn.Module):
        def __init__(self, input_dim, output_dim, hidden_dims, activation_fn):
            super(policyNetwork, self).__init__()
            self.activation_fn = activation_fn

            # Device management
            self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

            # Initialize layers
            self.layers = nn.ModuleList()
            self.layers.append(nn.Linear(input_dim, hidden_dims[0]))
            for i in range(len(hidden_dims)-1):
                self.layers.append(nn.Linear(hidden_dims[i], hidden_dims[i+1]))
            self.layers.append(nn.Linear(hidden_dims[-1], output_dim))

        def _format(self, x):
            if not isinstance(x, torch.Tensor):
                x = torch.tensor(x, dtype=torch.float32)
                x = x.unsqueeze(0)  # Add batch dimension if missing
            return x

        def forward(self, x):
            x = self._format(x)
            for layer in self.layers[:-1]:
                x = self.activation_fn(layer(x))
            return self.layers[-1](x)  # No activation on the output layer

        def full_pass(self, x):
            logits = self.forward(x)
            dist = torch.distributions.Categorical(logits=logits)
            action = dist.sample()
            log_prob_action = dist.log_prob(action).unsqueeze(-1)
            entropy = dist.entropy().unsqueeze(-1)
            is_exploratory = action != np.argmax(logits.detach().numpy())
            return action.item(), is_exploratory.item(), log_prob_action, entropy

        def select_action(self, x):
            logits = self.forward(x)
            dist = torch.distributions.Categorical(logits=logits)
            action = dist.sample()
            return action.item()

        def select_greedy_action(self, x):
            logits = self.forward(x)
            return np.argmax(logits.detach().numpy())

    return policyNetwork(inDim, outDim, hDim, activation)


In [ ]:
#Policy network for VPG
def createPolicyNetworkVPG(inDim, outDim, hDim = [32,32], activation = F.relu):

  class policyNetwork(nn.Module):

    def __init__(self, input_size, layer_sizes=(32, 32), activation_function=F.relu):
        """
        Initialize the policy network with specified input size, hidden layer sizes,
        and activation function. The network architecture includes a sequence of
        fully connected layers.

        Parameters:
        - input_size: Integer, size of the input layer.
        - layer_sizes: Tuple of integers, sizes of the hidden layers.
        - activation_function: Callable, the activation function to use in the network.
        """
        super(policyNetwork, self).__init__()
        self.activate = activation_function

        self.first_layer = nn.Linear(input_size, layer_sizes[0])
        self.middle_layers = nn.ModuleList()
        for i in range(len(layer_sizes) - 1):
            layer = nn.Linear(layer_sizes[i], layer_sizes[i + 1])
            self.middle_layers.append(layer)
        self.final_layer = nn.Linear(layer_sizes[-1], 1)

    def _format(self, input_state):
        """
        Ensures the input is in the correct torch.Tensor format.

        Parameters:
        - input_state: The state input to the network, can be a list or a numpy array.

        Returns:
        - A torch.Tensor of the state, ready for processing by the network.
        """
        tensor_input = input_state
        if not isinstance(tensor_input, torch.Tensor):
            tensor_input = torch.tensor(tensor_input, dtype=torch.float32)
            tensor_input = tensor_input.unsqueeze(0)
        return tensor_input

    def forward(self, state):
        """
        Forward pass through the network.

        Parameters:
        - state: The input state to the network.

        Returns:
        - The output from the network.
        """
        state = self._format(state)
        state = self.activate(self.first_layer(state))
        for layer in self.middle_layers:
            state = self.activate(layer(state))
        return self.final_layer(state)

  return policyNetwork(inDim, hDim, activation)



In [ ]:
def plotQuantity(quantityListDict, totalEpisodeCount, descriptionList):
    #this function takes in the quantityListDict and plots quantity vs episodes.
    #quantityListListDict = {envInstanceCount: quantityList}
    #quantityList is list of the qunatity per episode,
    #for example it could be mean reward per episode, traintime per episode, etc.
    #
    #NOTE: len(quantityList) == totalEpisodeCount
    #
    #Since we run multiple instances of the environment, there will be variance across environments
    #so in the plot, you will plot per episode maximum, minimum and average value across all env instances
    #Basically, you need to envelop (e.g., via color) the quantity between max and min with mean value in between
    #
    #use the descriptionList parameter to put legends, title, etc.
    #For each of the plot, create the legend on the left/right side so that it doesn't overlay on the plot lines/envelop.
    #
    #this is a generic function and can be used to plot any of the quantity of interest
    #In particular we will be using this function to plot:
    #        mean train rewards vs episodes
    #        mean evaluation rewards vs episodes
    #        total steps vs episode
    #        train time vs episode
    #        wall clock time vs episode
    #
    #
    #this function doesn't return anything

    #Your code goes in here


    return

In [ ]:
def huberLoss(error, delta):
    #this function calculates the huber loss for the error using the delta parameter

    #Your code goes in here

    return hLoss

In [ ]:
#in case you want to create any other helper function, the code goes in here
def reset_env(env, seed=None, options=None):
    # Reset the environment with seed and options, handling new seeding mechanism
    initial_observation, info = env.reset(seed=seed, options=options)
    return initial_observation, info

In [ ]:
#in case you want to create any other helper function, the code goes in here

# Deep Value Based RL agents.
<a id="deep-value-based"></a>

### The purpose of this part is to learn about different Deep Value Based RL agents.

In this part of the assignment you will be implementing Deep RL algorithms we learnt in Lectures. Namely, we will be implementing NFQ, DQN, Double DQN (DDQN), Duelling Double DQN (D3QN), and Duelling Double DQN with Prioritized Experience Replay (D3QN-PER). For all the algorithms below, this time we will not be specifying the hyper-parameters, please play with the hyper-params to come up with the best values. This way you will learn to tune the model. Some of the values were specified in the lecture, that would be a good starting point. Your aim is to develop the best NFQ/DQN/DDQN/D3QN/D3QN-PER agent for each of the setting.  

For those of you who follow TEDEd, here is an interesting video by TED on DQN and Atari Games: https://www.youtube.com/watch?v=PP8Zc778B8s

Also since these environments are available in Gymanasium, there are public leaderboards (https://github.com/openai/gym/wiki/Leaderboard) for each of these environments. Compare where does your agent stand on these leaderboard for each of these environments, try to tune your agents so that it is on the top of the leaderboard. In fact, if your agent performs well on these environments, you can alse make your entry on the leaderboard.  

## <font color='green'> Do not change any Class/Methods definition. We have split the class methods across cells for code readibility purposes. This requires to inherit the same class, please do not change it. </font>

## ReplayBuffer

In next few cells, you will implement replaybuffer class.

This class creates a buffer for storing and retrieving experiences. This is a generic class and can be used
for different agents like NFQ, DQN, DDQN, PER_DDQN, etc.
Following are the methods for this class which are implemented in subsequent cells

```
class ReplayBuffer():
    def __init__(self, bufferSize, **kwargs)
    def store(self, experience)
    def update(self, indices, priorities)
    def collectExperiences(env, state, explorationStrategy, net = None)
    def sample(self, batchSize, **kwargs)
    def splitExperiences(self, experiences)
    def length(self)
```   

In [ ]:
class ReplayBuffer():
    def __init__(self, bufferSize, bufferType = 'DQN', **kwargs):
        # this function creates the relevant data-structures, and intializes all relevant variables
        # it can take variable number of parameters like alpha, beta, beta_rate (required for PER)
        # here the bufferType variable can be used to maintain one class for all types of agents
        # using the bufferType parameter in the methods below, you can implement all possible functionalities
        # that could be used for different types of agents
        # permissible values for bufferType = NFQ, DQN, DDQN, D3QN and PER-D3QN

        #Your code goes in here
        self.ss_mem = np.empty(shape=(bufferSize), dtype=np.ndarray)
        self.as_mem = np.empty(shape=(bufferSize), dtype=np.ndarray)
        self.rs_mem = np.empty(shape=(bufferSize), dtype=np.ndarray)
        self.ps_mem = np.empty(shape=(bufferSize), dtype=np.ndarray)
        self.ds_mem = np.empty(shape=(bufferSize), dtype=np.ndarray)

        self.max_size = bufferSize
        self.bufferType = bufferType
        self._idx = 0
        self.size = 0
        # Assume default batch size for sampling if not provided
        self.default_batch_size = 64

        #kwargs
        if(self.bufferType == "D3QN-PER"):
            
            self.max_samples = kwargs.get('max_samples', 10000)
            self.batch_size = kwargs.get('batch_size', 64)
            self.rank_based = kwargs.get('rank_based', False)
            self.alpha = kwargs.get('alpha', 0.6)  
            self.beta = kwargs.get('beta', 0.1)  
            self.beta_rate = kwargs.get('beta_rate', 0.9999)
            
            # Other initializations that depend on the parameters above
            self.memory = np.empty(shape=(self.max_samples, 2), dtype=np.ndarray)
            self.n_entries = 0
            self.next_index = 0
            self.td_error_index = 0
            self.sample_index = 1


        return

In [ ]:
class ReplayBuffer(ReplayBuffer):
    def store(self, experience):
        #stores the experiences, based on parameters in init it can assign priorities, etc.
        #
        #this function does not return anything
        #
        #Your code goes in here
        if(self.bufferType == "D3QN-PER"):
            priority = 1.0
            if self.n_entries > 0:
                priority = self.memory[
                    :self.n_entries, 
                    self.td_error_index].max()
            self.memory[self.next_index, 
                        self.td_error_index] = priority
            self.memory[self.next_index, 
                        self.sample_index] = np.array(experience, dtype = 'object')
            self.n_entries = min(self.n_entries + 1, self.max_samples)
            self.next_index += 1
            self.next_index = self.next_index % self.max_samples

        else:
            
            s, a, r, p, d = experience
            self.ss_mem[self._idx] = s
            self.as_mem[self._idx] = a
            self.rs_mem[self._idx] = r
            self.ps_mem[self._idx] = p
            self.ds_mem[self._idx] = d
    
            self._idx = (self._idx + 1) % self.max_size
            self.size = min(self.size + 1, self.max_size)

        return

In [ ]:
class ReplayBuffer(ReplayBuffer):
    def update(self, idxs, td_errors):
        self.memory[idxs, self.td_error_index] = np.abs(td_errors)
        if self.rank_based:
            sorted_arg = self.memory[:self.n_entries, self.td_error_index].argsort()[::-1]
            self.memory[:self.n_entries] = self.memory[sorted_arg]

    def _update_beta(self):
        self.beta = min(1.0, self.beta * self.beta_rate**-1)
        return self.beta

In [ ]:
class ReplayBuffer(ReplayBuffer):
    def collectExperiences(env, state, explorationStrategy, countExperiences, net = None):
        #this method allows the agent to interact with the environment starting from a state and it collects
        #experiences during the interaction, it uses network to get the value function and uses exploration strategy
        #to select action. It collects countExperiences and in case the environment terminates before that it returns
        #the function calling this method needs to handle early termination accordingly.
        #
        #this function does not return anything
        #
        #Your code goes in here
        experiences = []
        for _ in range(countExperiences):
            action = explorationStrategy(state, net)  # Assuming explorationStrategy uses net
            next_state, reward, done, _ = env.step(action)
            experiences.append((state, action, reward, next_state, done))
            if done:
                break
            state = next_state
        return experiences



In [ ]:
class ReplayBuffer(ReplayBuffer):
    def sample(self, batchSize=None, **kwargs):
        # this method returns batchSize number of experiences
        # based on extra arguments, it could do sampling or it could return the latest batchSize experiences or
        # via some other strategy
        #
        # in the case of Prioritized Experience Replay (PER) the sampling needs to take into account the priorities
        #
        # this function returns experiences samples
        #
        #Your code goes in here
        if(self.bufferType == "D3QN-PER"):
            batchSize = self.batch_size if batchSize == None else batchSize
            self._update_beta()
            entries = self.memory[:self.n_entries]
    
            if self.rank_based:
                priorities = 1/(np.arange(self.n_entries) + 1)
            else: # proportional
                priorities = entries[:, self.td_error_index] 
            scaled_priorities = priorities**self.alpha        
            probs = np.array(scaled_priorities/np.sum(scaled_priorities), dtype=np.float64)
    
            weights = (self.n_entries * probs)**-self.beta
            normalized_weights = weights/weights.max()
            idxs = np.random.choice(self.n_entries, batchSize, replace=False, p=probs)
            samples = np.array([entries[idx] for idx in idxs])
            
            samples_stacks = [np.vstack(batch_type) for batch_type in np.vstack(samples[:, self.sample_index]).T]
            idxs_stack = np.vstack(idxs)
            weights_stack = np.vstack(normalized_weights[idxs])
            return idxs_stack, weights_stack, samples_stacks   

        else:
        
            if batchSize is None:
                batchSize = self.default_batch_size
            idxs = np.random.choice(self.size, batchSize, replace=False)
            experiences = (np.vstack(self.ss_mem[idxs]), \
                           np.vstack(self.as_mem[idxs]), \
                           np.vstack(self.rs_mem[idxs]), \
                           np.vstack(self.ps_mem[idxs]), \
                           np.vstack(self.ds_mem[idxs]))
            return experiences




In [ ]:
class ReplayBuffer(ReplayBuffer):
    def splitExperiences(self, experiences):
        #it takes in experiences and gives the following:
        #states, actions, rewards, nextStates, dones
        #
        #Your code goes in here
        states, actions, rewards, nextStates, dones = experiences
        return states, actions, rewards, nextStates, dones



In [ ]:
class ReplayBuffer(ReplayBuffer):
    def length(self):
        #tells the number of experiences stored in the internal buffer
        if(self.bufferType == "D3QN-PER"):
            return self.n_entries
        return self.size

## Neural Fitted Q (NFQ)

Implement the Neural Fitted Q algorithm. We have studied about NFQ algorithm in the Lecture. Use the function definitions (given below).

This class implements the NFQ Agent, you are required to implement the various methods of this class
as outlined below. Note this class is generic and should work with any permissible Gym environment.
Also please feel free to play with different exploration strategies with decaying paramters (epsilon/temperature)

```
class NFQ():
    def __init__(env, seed, gamma, epochs,
                 bufferSize,
                 batchSize,
                 optimizerFn,
                 optimizerLR,
                 MAX_TRAIN_EPISODES, MAX_EVAL_EPISODES,
                 explorationStrategyTrainFn,
                 explorationStrategyEvalFn)
    def initBookKeeping(self)
    def performBookKeeping(self, train = True)
    def runNFQ(self)
    def trainAgent(self)
    def trainNetwork(self, experiences, epochs)
    def evaluateAgent(self)
```

In [ ]:
class NFQ():
    def __init__(self, env_id, seed, gamma, epochs,
                 bufferSize,
                 batchSize,
                 optimizerFn,
                 optimizerLR,
                 MAX_TRAIN_EPISODES, MAX_EVAL_EPISODES,
                 explorationStrategyTrainFn,
                 explorationStrategyEvalFn):

        #this NFQ method
        # 1. creates and initializes (with seed) the environment, train/eval episodes, gamma, etc.
        # 2. creates and intializes all the variables required for book-keeping values via the initBookKeeping method
        # 3. creates Q-network using the createValueNetwork above
        # 4. creates and initializes (with network params) the optimizer function
        # 5. sets the explorationStartegy variables/functions for train and evaluation
        # 6. sets the batchSize for the number of experiences
        # 7. Creates the replayBuffer

        # 1. Environment initialization using env_id
        self.env = gym.make(env_id)
        self.seed = seed
        reset_env(self.env, seed=seed)  # Seed the environment (affects environment operations)

        # Note on seeding:
        # Seeding affects global state for NumPy and PyTorch, which might impact other parts of an application.
        # It's set here for reproducibility within the scope of this NFQ instance's operations.
        torch.manual_seed(seed)  # Seed PyTorch (global effect)
        np.random.seed(seed)  # Seed NumPy (global effect)
        random.seed(seed)

        # 2. Book-keeping initialization
        #self.initBookKeeping()

        # 3. Creating Q-network
        inDim = self.env.observation_space.shape[0]
        outDim = self.env.action_space.n
        hDim = [512,128]  # Example hidden layer dimensions, adjust as needed
        activation = torch.nn.functional.relu  # Specify the activation function
        self.q_network = createValueNetwork(inDim, outDim, hDim, activation)

        # 4. Optimizer initialization
        self.optimizer = optimizerFn(self.q_network, optimizerLR)

        # 5. Exploration strategy setup
        self.explorationStrategyTrain = explorationStrategyTrainFn
        self.explorationStrategyEval = explorationStrategyEvalFn

        # 6. Other variables
        self.gamma = gamma
        self.epochs = epochs
        self.batchSize = batchSize
        self.MAX_TRAIN_EPISODES = MAX_TRAIN_EPISODES
        self.MAX_EVAL_EPISODES = MAX_EVAL_EPISODES






In [ ]:
class NFQ(NFQ):
    def initBookKeeping(self):
        #this method creates and intializes all the variables required for book-keeping values and it is called by
        #init method
        pass

    #Your code goes in here

In [ ]:
class NFQ(NFQ):
    def performBookKeeping(self, train = True):
        #this method updates relevant variables for the bookKeeping, this can be called
        #multiple times during training
        #if you want you can print information using this, so it may help to monitor progress and also help to debug
        pass

    #Your code goes in here

In [ ]:
class NFQ(NFQ):
        def stepint(self, state, env):
          action = self.explorationStrategyTrain(self.online_model, state, 0.5) #0.5 is the epsilon value
          new_state, reward, is_terminal, is_truncated, info = env.step(action)
          is_failure = is_terminal and not is_truncated
          experience = (state, action, reward, new_state, float(is_failure))

          self.experiences.append(experience)
          self.episode_reward[-1] += reward
          self.episode_timestep[-1] += 1
          #self.episode_exploration[-1] += int(self.training_strategy.exploratory_action_taken)
          return new_state, is_terminal

In [ ]:
class NFQ(NFQ):
    def runNFQ(self):
        #this is the main method, it trains the agent, performs bookkeeping while training and finally evaluates
        #the agent and returns the following quantities:
        #1. episode wise mean train rewards
        #2. epsiode wise mean eval rewards
        #2. episode wise trainTime (in seconds): time elapsed during training since the start of the first episode
        #3. episode wise wallClockTime (in seconds): actual time elapsed since the start of training,
        #                               note this will include time for BookKeeping and evaluation
        # Note both trainTime and wallClockTime get accumulated as episodes proceed.


        #Your code goes in here
        resultList, trainTimeList, evalRewardsList, wallClockTimeList = self.trainAgent()
        resultEval = self.evaluateAgent()
        finalEvalReward  = np.mean(resultEval)


        return resultList, trainTimeList, evalRewardsList, wallClockTimeList, finalEvalReward

In [ ]:
class NFQ(NFQ):
    def trainAgent(self):


        #this method collects experiences and trains the NFQ agent and does BookKeeping while training.
        #this calls the trainNetwork() method internally, it also evaluates the agent per episode
        #it trains the agent for MAX_TRAIN_EPISODES

        torch.manual_seed(self.seed) ; np.random.seed(self.seed) ; random.seed(self.seed)
        self.episode_timestep = []
        self.episode_reward = []
        self.episode_seconds = []
        self.evaluation_scores = []
        self.episode_exploration = []
        training_start = time.time()
        global train_count
        train_count +=1
        self.online_model = self.q_network
        self.value_optimizer = self.optimizer
        self.experiences = []
        max_episodes = self.MAX_TRAIN_EPISODES
        result = np.empty((2000, 5))
        result[:] = np.nan
        training_time = 0

        for episode in range(1, max_episodes + 1):
            print(f'Episode {episode} and train called {train_count} times')
            episode_start = time.time()

            state, _ = self.env.reset(seed=self.seed)
            self.episode_reward.append(0.0)
            self.episode_timestep.append(0.0)
            self.episode_exploration.append(0.0)

            for step in count():
                state, is_terminal = self.stepint(state, self.env)

                if len(self.experiences) >= self.batchSize:

                  '''
                  #debug going on
                    #print(self.experiences.shape)
                    #for element in self.experiences:
                      #print(element.shape)
                    print(self.experiences)
                    states, actions, rewards, next_states, done_flags = [], [], [], [], []
                    for state, action, reward, next_state, done in self.experiences:
                      states.append(state)
                      actions.append(action)
                      rewards.append(reward)
                      next_states.append(next_state)
                      done_flags.append(done)

                    experiences = np.array([states, actions, rewards, next_states, done_flags])
                  #debug went on
                  '''
                  experiences = np.array(self.experiences, dtype="object")
                  batches = [np.vstack(sars) for sars in experiences.T]
                  experiences = self.online_model.load(batches)
                  self.trainNetwork(experiences,self.epochs)
                  self.experiences.clear()

                if is_terminal:
                    #gc.collect()
                    break

            episode_elapsed = time.time() - episode_start
            self.episode_seconds.append(episode_elapsed)
            training_time += episode_elapsed
            evaluation_score= np.mean(self.evaluateAgent())

            total_step = int(np.sum(self.episode_timestep))
            self.evaluation_scores.append(evaluation_score)

            mean_10_reward = np.mean(self.episode_reward[-10:])
            std_10_reward = np.std(self.episode_reward[-10:])
            mean_100_reward = np.mean(self.episode_reward[-100:])
            std_100_reward = np.std(self.episode_reward[-100:])
            mean_100_eval_score = np.mean(self.evaluation_scores[-100:])
            std_100_eval_score = np.std(self.evaluation_scores[-100:])
            #lst_100_exp_rat = np.array(self.episode_exploration[-100:])/np.array(self.episode_timestep[-100:])
            #mean_100_exp_rat = np.mean(lst_100_exp_rat)
            #std_100_exp_rat = np.std(lst_100_exp_rat)

            wallclock_elapsed = time.time() - training_start
            result[episode-1] = total_step, mean_100_reward, mean_100_eval_score, training_time, wallclock_elapsed
            reached_max_minutes = wallclock_elapsed >= 10 * 60
            reached_max_episodes = episode >= 10000
            #reached_goal_mean_reward = mean_100_eval_score >= 475
            reached_goal_mean_reward = False
            training_is_over = reached_max_minutes or reached_max_episodes or reached_goal_mean_reward
            if training_is_over:
                if reached_max_minutes: print(u'--> reached_max_minutes \u2715')
                if reached_max_episodes: print(u'--> reached_max_episodes \u2715')
                if reached_goal_mean_reward: print(u'--> reached_goal_mean_reward \u2713')
                break

        final_eval_rwd_list = self.evaluateAgent()
        mean_eval_rwd = np.mean(final_eval_rwd_list)
        wallclock_time = time.time() - training_start
        self.env.close()

        return result, training_time, final_eval_rwd_list, wallclock_time





In [ ]:
class NFQ(NFQ):
    def trainNetwork(self, experiences, epochs):

        for _ in range(epochs):

            states, actions, rewards, next_states, is_terminals = experiences
            batch_size = len(is_terminals)

            max_a_q_sp = self.online_model(next_states).detach().max(1)[0].unsqueeze(1)
            target_q_s = rewards + self.gamma * max_a_q_sp * (1 - is_terminals)
            q_sa = self.online_model(states).gather(1, actions)

            td_errors = q_sa - target_q_s
            value_loss = td_errors.pow(2).mul(0.5).mean()
            self.value_optimizer.zero_grad()
            value_loss.backward()
            self.value_optimizer.step()


        # this method trains the value network epoch number of times and is called by the trainAgent function
        # it essentially uses the experiences to calculate target, using the targets it calculates the error, which
        # is further used for calulating the loss. It then uses the optimizer over the loss
        # to update the params of the network by backpropagating through the network
        # this function does not return anything
        # you can try out other loss functions other than MSE like Huber loss, MAE, etc.


        return

In [ ]:
class NFQ(NFQ):
    def evaluateAgent(self):
        #this function evaluates the agent using the value network, it evaluates agent for MAX_EVAL_EPISODES
        #typcially MAX_EVAL_EPISODES = 1
        rwd_list = []
        for _ in range(self.MAX_EVAL_EPISODES):
            s, _ = self.env.reset(seed=self.seed)
            rwd_list.append(0)
            for _ in count():
                a = self.explorationStrategyEval(self.online_model, s)
                s, rwd, done, truncated,_ = self.env.step(a)
                rwd_list[-1] += rwd
                if done or truncated: break
        return rwd_list
        #return np.mean(rwd_list), np.std(rwd_list)
        #return finalEvalRewardsList  can we return rs from here and do np.mean and std later?

In [ ]:
train_count = 0
nfq_results = []
best_agent, best_eval_score = None, float('-inf')
seed_list = [420,133,74,317,233]
for myseed in seed_list:

    batch_size = 1024
    epochs = 40
    val_optimizer_fn = lambda net, lr: optim.RMSprop(net.parameters(), lr=lr)

    env_name = 'CartPole-v1'
    gamma = 0.99
    max_episodes = 10000
    goal_mean_100_reward = 475

    nfq_instance = NFQ(
    env_id='CartPole-v1',
    seed=myseed,
    gamma=0.99,
    epochs=30,
    bufferSize=10000,
    batchSize=1024,
    optimizerFn=val_optimizer_fn,
    optimizerLR=1e-3,
    MAX_TRAIN_EPISODES=2000,
    MAX_EVAL_EPISODES=1,
    explorationStrategyTrainFn=selectEpsilonGreedyAction,
    explorationStrategyEvalFn=selectGreedyAction)

    trainRewardsList, trainTimeList, evalRewardsList, wallClockTimeList, final_eval_score = nfq_instance.runNFQ()
    nfq_results.append(trainRewardsList)
    if final_eval_score > best_eval_score:
        best_eval_score = final_eval_score
        best_agent = nfq_instance

nfq_results = np.array(nfq_results)

In [ ]:
reward_max, steps_max, eval_score_max, train_time_max, wall_clock_max = np.max(nfq_results, axis=0).T
reward_min, steps_min, eval_score_min, train_time_min, wall_clock_min = np.min(nfq_results, axis=0).T
reward_avg, steps_avg, eval_score_avg, train_time_avg, wall_clock_avg = np.mean(nfq_results, axis=0).T
episode_indices = np.arange(len(reward_avg))

plt.style.use('ggplot')  
fig, plot_areas = plt.subplots(5, 1, figsize=(12, 25), sharex='col')
fig.subplots_adjust(hspace=0.5) 

colors = ['blue', 'green', 'red', 'purple', 'orange']
titles = ['Total Steps','Training Reward', 'Evaluation reward',  'Training Duration', 'Wall-clock Time']
y_labels = ['Steps', 'Reward', 'Reward', 'Seconds', 'Seconds']
data_max = [reward_max, steps_max, eval_score_max, train_time_max, wall_clock_max]
data_min = [reward_min, steps_min, eval_score_min, train_time_min, wall_clock_min]
data_avg = [reward_avg, steps_avg, eval_score_avg, train_time_avg, wall_clock_avg]

# Generate plots
for ax, title, color, max_data, min_data, avg_data, y_label in zip(plot_areas, titles, colors, data_max, data_min, data_avg, y_labels):
    ax.plot(max_data, linestyle='--', color=color, alpha=0.75)
    ax.plot(min_data, linestyle='--', color=color, alpha=0.75)
    ax.plot(avg_data, label='NFQ', color=color, linewidth=2)
    ax.fill_between(episode_indices, min_data, max_data, color=color, alpha=0.2)
    ax.set_title(title)
    ax.set_ylabel(y_label)
    ax.legend()

plot_areas[-1].set_xlabel('Episodes')
plt.show()


## Deep Q-Network (DQN)

Implement the Deep Q algorithm. We have studied about DQN algorithm in the Lecture. Use the function definitions (given below).

This class implements the DQN Agent, you are required to implement the various methods of this class
as outlined below. Note this class is generic and should work with any permissible Gym environment

```
class DQN():
    def __init__(env, seed, gamma,
                 bufferSize,
                 batchSize,
                 optimizerFn,
                 optimizerLR,
                 MAX_TRAIN_EPISODES, MAX_EVAL_EPISODES,
                 explorationStrategyTrainFn,
                 explorationStrategyEvalFn,
                 updateFrequency)
    def initBookKeeping(self)
    def performBookKeeping(self, train = True)
    def runDQN(self)
    def trainAgent(self)
    def trainNetwork(self, experiences)
    def updateNetwork(self, onlineNet, targetNet)
    def evaluateAgent(self)
```

In [ ]:
class DQN():
    def __init__(self, env_id, seed, gamma,epochs,
                 bufferSize,
                 batchSize,
                 optimizerFn,
                 optimizerLR,
                 MAX_TRAIN_EPISODES, MAX_EVAL_EPISODES,
                 explorationStrategyTrainFn,
                 explorationStrategyEvalFn,
                 updateFrequency):
        #this DQN method
        # 1. creates and initializes (with seed) the environment, train/eval episodes, gamma, etc.
        # 2. creates and intializes all the variables required for book-keeping values via the initBookKeeping method
        # 3. creates traget and online Q-networks using the createValueNetwork above
        # 4. creates and initializes (with network params) the optimizer function
        # 5. sets the explorationStartegy variables/functions for train and evaluation
        # 6. sets the batchSize for the number of experiences
        # 7. Creates the replayBuffer

        # 1. Environment initialization using env_id

        self.env = gym.make(env_id)
        self.seed = seed
        reset_env(self.env, seed=seed)  # Seed the environment (affects environment operations)

        # Note on seeding:
        # Seeding affects global state for NumPy and PyTorch, which might impact other parts of an application.
        # It's set here for reproducibility within the scope of this NFQ instance's operations.
        torch.manual_seed(seed)  # Seed PyTorch (global effect)
        np.random.seed(seed)  # Seed NumPy (global effect)
        random.seed(seed)

        # 2. Book-keeping initialization
        #self.initBookKeeping()

        # 3. Creating online Q-network
        inDim = self.env.observation_space.shape[0]
        outDim = self.env.action_space.n
        hDim = [512,128]  # Example hidden layer dimensions, adjust as needed
        activation = torch.nn.functional.relu  # Specify the activation function
        self.target_model = createValueNetwork(inDim, outDim, hDim, activation)
        self.online_model = createValueNetwork(inDim, outDim, hDim, activation)

        # 4. Optimizer initialization
        self.optimizer = optimizerFn(self.online_model, optimizerLR)

        # 5. Exploration strategy setup
        self.explorationStrategyTrain = explorationStrategyTrainFn
        self.explorationStrategyEval = explorationStrategyEvalFn

        # 6. Other variables
        self.gamma = gamma
        self.epochs = epochs
        self.batchSize = batchSize
        self.MAX_TRAIN_EPISODES = MAX_TRAIN_EPISODES
        self.MAX_EVAL_EPISODES = MAX_EVAL_EPISODES
        self.updateFrequency = updateFrequency

        # 7. Replay buffer
        self.replay_buffer_fn = lambda: ReplayBuffer(bufferSize=50000)



In [ ]:
class DQN(DQN):
    def initBookKeeping(self):
        #this method creates and intializes all the variables required for book-keeping values and it is called
        #init method
        #
        # Your code goes in here
        #

        return

In [ ]:
class DQN(DQN):
    def performBookKeeping(self, train = True):
        #this method updates relevant variables for the bookKeeping, this can be called
        #multiple times during training
        #if you want you can print information using this, so it may help to monitor progress and also help to debug
        #
        # Your code goes in here
        #

        return

In [ ]:
class DQN(DQN):
        def stepint(self, state, env):
          action = self.explorationStrategyTrain(self.online_model, state)
          new_state, reward, is_terminal, is_truncated, info = env.step(action)
          is_failure = is_terminal and not is_truncated
          experience = (state, action, reward, new_state, float(is_failure))

          self.replay_buffer.store(experience)
          self.episode_reward[-1] += reward
          self.episode_timestep[-1] += 1
          return new_state, is_terminal


In [ ]:
class DQN(DQN):
    def runDQN(self):
        #this is the main method, it trains the agent, performs bookkeeping while training and finally evaluates
        #the agent and returns the following quantities:
        #1. episode wise mean train rewards
        #2. epsiode wise mean eval rewards
        #2. episode wise trainTime (in seconds): time elapsed during training since the start of the first episode
        #3. episode wise wallClockTime (in seconds): actual time elapsed since the start of training,
        #                               note this will include time for BookKeeping and evaluation
        # Note both trainTime and wallClockTime get accumulated as episodes proceed.
        #
        #Your code goes in here
        resultList, trainTimeList, evalRewardsList, wallClockTimeList = self.trainAgent()
        resultEval = self.evaluateAgent()
        finalEvalReward  = np.mean(resultEval)


        return resultList, trainTimeList, evalRewardsList, wallClockTimeList, finalEvalReward



In [ ]:
class DQN(DQN):
    def trainAgent(self):
        # this method collects experiences and trains the agent and does BookKeeping while training.
        # this calls the trainNetwork() method internally, it also evaluates the agent per episode
        # it trains the agent for MAX_TRAIN_EPISODES
        training_start = time.time()
        torch.manual_seed(self.seed)
        np.random.seed(self.seed)
        random.seed(self.seed)
        self.episode_timestep = []
        self.episode_reward = []
        self.episode_seconds = []
        self.evaluation_scores = []
        self.episode_exploration = []
        global train_count
        train_count +=1

        self.updateNetwork()
        self.value_optimizer = self.optimizer

        self.replay_buffer = self.replay_buffer_fn()
        tempstate, _ = self.env.reset(seed=self.seed)
        self.explorationStrategyTrain(self.online_model, tempstate, init=True) #IMPORTANT

        max_episodes = self.MAX_TRAIN_EPISODES
        result = np.empty((2000, 5))
        result[:] = np.nan
        training_time = 0

        for episode in range(1, max_episodes + 1):
            episode_start = time.time()
            print(f'Episode {episode} and train called {train_count} times')
            state, _ = self.env.reset(seed=self.seed)
            self.episode_reward.append(0.0)
            self.episode_timestep.append(0.0)
            self.episode_exploration.append(0.0)

            for step in count():
                state, is_terminal = self.stepint(state, self.env)
                min_samples = self.replay_buffer.default_batch_size * self.batchSize
                if self.replay_buffer.length() > min_samples:
                    experiences = self.replay_buffer.sample()
                    experiences = self.online_model.load(experiences)
                    self.trainNetwork(experiences, self.epochs)

                if np.sum(self.episode_timestep) % self.updateFrequency == 0:
                    self.updateNetwork()

                if is_terminal:
                    break

            episode_elapsed = time.time() - episode_start
            self.episode_seconds.append(episode_elapsed)
            training_time += episode_elapsed
            evaluation_score = np.mean(self.evaluateAgent())
            total_step = int(np.sum(self.episode_timestep))
            self.evaluation_scores.append(evaluation_score)

            mean_10_reward = np.mean(self.episode_reward[-10:])
            std_10_reward = np.std(self.episode_reward[-10:])
            mean_100_reward = np.mean(self.episode_reward[-100:])
            std_100_reward = np.std(self.episode_reward[-100:])
            mean_100_eval_score = np.mean(self.evaluation_scores[-100:])
            std_100_eval_score = np.std(self.evaluation_scores[-100:])

            wallclock_elapsed = time.time() - training_start
            result[episode - 1] = total_step, mean_100_reward, mean_100_eval_score, training_time, wallclock_elapsed
            reached_max_minutes = wallclock_elapsed >= 10 * 60
            reached_max_episodes = episode >= 10000
            reached_goal_mean_reward = mean_100_eval_score >= 475
            #reached_goal_mean_reward=  False
            training_is_over = reached_max_minutes or reached_max_episodes or reached_goal_mean_reward
            if training_is_over:
                if reached_max_minutes:
                    print(u'--> reached_max_minutes ✕')
                if reached_max_episodes:
                    print(u'--> reached_max_episodes ✕')
                if reached_goal_mean_reward:
                    print(u'--> reached_goal_mean_reward ✓')
                break

        final_eval_rwd_list = self.evaluateAgent()
        mean_eval_rwd = np.mean(final_eval_rwd_list)
        wallclock_time = time.time() - training_start
        self.env.close()

        return result, training_time, final_eval_rwd_list, wallclock_time


In [ ]:
class DQN(DQN):
    def trainNetwork(self, experiences, epochs):
        # this method trains the value network epoch number of times and is called by the trainAgent function
        # it essentially uses the experiences to calculate target, using the targets it calculates the error, which
        # is further used for calulating the loss. It then uses the optimizer over the loss
        # to update the params of the network by backpropagating through the network
        # this function does not return anything
        # you can try out other loss functions other than MSE like Huber loss, MAE, etc.
        #
        #Your code goes in here
        states, actions, rewards, next_states, is_terminals = experiences
        batch_size = len(is_terminals)

        max_a_q_sp = self.target_model(next_states).detach().max(1)[0].unsqueeze(1)
        target_q_sa = rewards + (self.gamma * max_a_q_sp * (1 - is_terminals))
        q_sa = self.online_model(states).gather(1, actions)

        td_error = q_sa - target_q_sa
        value_loss = td_error.pow(2).mul(0.5).mean()
        self.value_optimizer.zero_grad()
        value_loss.backward()
        self.value_optimizer.step()

        return

In [ ]:
class DQN(DQN):
    def updateNetwork(self):
        #this function updates the onlineNetwork with the target network
        for target, online in zip(self.target_model.parameters(),
                                  self.online_model.parameters()):
            target.data.copy_(online.data)


In [ ]:
class DQN(DQN):
    def evaluateAgent(self):
        #this function evaluates the agent using the value network, it evaluates agent for MAX_EVAL_EPISODES
        #typcially MAX_EVAL_EPISODES = 1
        rwd_list = []
        for _ in range(self.MAX_EVAL_EPISODES):
            s, _ = self.env.reset(seed=self.seed)
            rwd_list.append(0)
            for _ in count():
                a = self.explorationStrategyEval(self.online_model, s)
                s, rwd, done, truncated,_ = self.env.step(a)
                rwd_list[-1] += rwd
                if done or truncated: break
        return rwd_list



In [ ]:
train_count = 0
dqn_results = []
best_agent, best_eval_score = None, float('-inf')
seed_list = [420, 133, 74, 317, 233]

for myseed in seed_list:

    val_optimizer_fn = lambda net, lr: optim.RMSprop(net.parameters(), lr=lr)

    # Instantiation of the DQN class
    dqn_instance = DQN(
        env_id='CartPole-v1',
        seed=myseed,
        gamma=0.99,
        epochs = 20,
        bufferSize=50000,
        batchSize=5,
        optimizerFn=val_optimizer_fn,
        optimizerLR=1e-3,
        MAX_TRAIN_EPISODES=300,
        MAX_EVAL_EPISODES=1,
        explorationStrategyTrainFn=selectEpsilonExpGreedyAction,
        explorationStrategyEvalFn=selectGreedyAction,
        updateFrequency=10
    )

    # Running the NFQ method and appending results
    trainRewardsList, trainTimeList, evalRewardsList, wallClockTimeList, final_eval_score = dqn_instance.runDQN()
    dqn_results.append(trainRewardsList)
    if final_eval_score > best_eval_score:
        best_eval_score = final_eval_score
        best_agent = dqn_instance

# Convert dqn_results to a numpy array for any further processing
dqn_results = np.array(dqn_results)

In [ ]:
reward_max, steps_max, eval_score_max, train_time_max, wall_clock_max = np.max(dqn_results, axis=0).T
reward_min, steps_min, eval_score_min, train_time_min, wall_clock_min = np.min(dqn_results, axis=0).T
reward_avg, steps_avg, eval_score_avg, train_time_avg, wall_clock_avg = np.mean(dqn_results, axis=0).T
episode_indices = np.arange(len(reward_avg))

plt.style.use('ggplot')  
fig, plot_areas = plt.subplots(5, 1, figsize=(12, 25), sharex='col')
fig.subplots_adjust(hspace=0.5)  
colors = ['blue', 'green', 'red', 'purple', 'orange']
titles = ['Total Steps','Training Reward', 'Evaluation reward',  'Training Duration', 'Wall-clock Time']
y_labels = ['Steps', 'Reward', 'Reward', 'Seconds', 'Seconds']
data_max = [reward_max, steps_max, eval_score_max, train_time_max, wall_clock_max]
data_min = [reward_min, steps_min, eval_score_min, train_time_min, wall_clock_min]
data_avg = [reward_avg, steps_avg, eval_score_avg, train_time_avg, wall_clock_avg]

# Generate plots
for ax, title, color, max_data, min_data, avg_data, y_label in zip(plot_areas, titles, colors, data_max, data_min, data_avg, y_labels):
    ax.plot(max_data, linestyle='--', color=color, alpha=0.75)
    ax.plot(min_data, linestyle='--', color=color, alpha=0.75)
    ax.plot(avg_data, label='DQN', color=color, linewidth=2)
    ax.fill_between(episode_indices, min_data, max_data, color=color, alpha=0.2)
    ax.set_title(title)
    ax.set_ylabel(y_label)
    ax.legend()

plot_areas[-1].set_xlabel('Episodes')
plt.show()


## Double DQN (DDQN)

Implement the Double DQN agent. We have studied about Double DQN agent in the Lecture. Use the function definitions (given below).

This class implements the Double DQN agent, you are required to implement the various methods of this class
as outlined below. Note this class is generic and should work with any permissible Gym environment

```
class DDQN():
    def __init__(env, seed, gamma,
                 bufferSize,
                 batchSize,
                 optimizerFn,
                 optimizerLR,
                 MAX_TRAIN_EPISODES, MAX_EVAL_EPISODES,
                 explorationStrategyTrainFn,
                 explorationStrategyEvalFn,
                 updateFrequency)
    def initBookKeeping(self)
    def performBookKeeping(self, train = True)
    def runDDQN(self)
    def trainAgent(self)
    def trainNetwork(self, experiences)
    def updateNetwork(self, onlineNet, targetNet)
    def evaluateAgent(self)
```

In [ ]:
class DDQN():
    def __init__(self,env_id, seed, gamma,epochs,
                 bufferSize,
                 batchSize,
                 optimizerFn,
                 optimizerLR,
                 max_gradient_norm,
                 MAX_TRAIN_EPISODES, MAX_EVAL_EPISODES,
                 explorationStrategyTrainFn,
                 explorationStrategyEvalFn,
                 updateFrequency):
        #this DDQN method
        # 1. creates and initializes (with seed) the environment, train/eval episodes, gamma, etc.
        # 2. creates and intializes all the variables required for book-keeping values via the initBookKeeping method
        # 3. creates tareget and online Q-networks using the createValueNetwork above
        # 4. creates and initializes (with network params) the optimizer function
        # 5. sets the explorationStartegy variables/functions for train and evaluation
        # 6. sets the batchSize for the number of experiences
        # 7. Creates the replayBuffer

        # 1. Environment initialization using env_id

        self.env = gym.make(env_id)
        self.seed = seed
        reset_env(self.env, seed=seed)  # Seed the environment (affects environment operations)

        # Note on seeding:
        # Seeding affects global state for NumPy and PyTorch, which might impact other parts of an application.
        # It's set here for reproducibility within the scope of this NFQ instance's operations.
        torch.manual_seed(seed)  # Seed PyTorch (global effect)
        np.random.seed(seed)  # Seed NumPy (global effect)
        random.seed(seed)

        # 2. Book-keeping initialization
        #self.initBookKeeping()

        # 3. Creating online Q-network
        inDim = self.env.observation_space.shape[0]
        outDim = self.env.action_space.n
        hDim = [512,128]  # Example hidden layer dimensions, adjust as needed
        activation = torch.nn.functional.relu  # Specify the activation function
        self.target_model = createValueNetwork(inDim, outDim, hDim, activation)
        self.online_model = createValueNetwork(inDim, outDim, hDim, activation)

        # 4. Optimizer initialization
        self.optimizer = optimizerFn(self.online_model, optimizerLR)

        # 5. Exploration strategy setup
        self.explorationStrategyTrain = explorationStrategyTrainFn
        self.explorationStrategyEval = explorationStrategyEvalFn

        # 6. Other variables
        self.gamma = gamma
        self.epochs = epochs
        self.batchSize = batchSize
        self.MAX_TRAIN_EPISODES = MAX_TRAIN_EPISODES
        self.MAX_EVAL_EPISODES = MAX_EVAL_EPISODES
        self.updateFrequency = updateFrequency
        self.max_gradient_norm = max_gradient_norm

        # 7. Replay buffer
        self.replay_buffer_fn = lambda: ReplayBuffer(bufferSize=50000)

In [ ]:
class DDQN(DDQN):
    def initBookKeeping(self):
        #this method creates and intializes all the variables required for book-keeping values and it is called
        #init method
        #
        # Your code goes in here
        #

        return

In [ ]:
class DDQN(DDQN):
    def performBookKeeping(self, train = True):
        #this method updates relevant variables for the bookKeeping, this can be called
        #multiple times during training
        #if you want you can print information using this, so it may help to monitor progress and also help to debug
        #
        # Your code goes in here
        #

        return

In [ ]:
class DDQN(DDQN):
    def stepint(self, state, env):

        action = self.explorationStrategyTrain(self.online_model, state)
        new_state, reward, is_terminal, is_truncated, info = env.step(action)
        is_failure = is_terminal and not is_truncated
        experience = (state, action, reward, new_state, float(is_failure))

        self.replay_buffer.store(experience)
        self.episode_reward[-1] += reward
        self.episode_timestep[-1] += 1
        #self.episode_exploration[-1] += int(self.training_strategy.exploratory_action_taken)
        return new_state, is_terminal

In [ ]:
class DDQN(DDQN):
    def runDDQN(self):
        #this is the main method, it trains the agent, performs bookkeeping while training and finally evaluates
        #the agent and returns the following quantities:
        #1. episode wise mean train rewards
        #2. epsiode wise mean eval rewards
        #2. episode wise trainTime (in seconds): time elapsed during training since the start of the first episode
        #3. episode wise wallClockTime (in seconds): actual time elapsed since the start of training,
        #                               note this will include time for BookKeeping and evaluation
        # Note both trainTime and wallClockTime get accumulated as episodes proceed.

        #Your code goes in here
        resultList, trainTimeList, evalRewardsList, wallClockTimeList = self.trainAgent()
        resultEval = self.evaluateAgent()
        finalEvalReward  = np.mean(resultEval)

        return resultList, trainTimeList, evalRewardsList, wallClockTimeList, finalEvalReward



In [ ]:
class DDQN(DDQN):
    def trainAgent(self):
        #this method collects experiences and trains the agent and does BookKeeping while training.
        #this calls the trainNetwork() method internally, it also evaluates the agent per episode
        #it trains the agent for MAX_TRAIN_EPISODES
        #
        training_start = time.time()
        torch.manual_seed(self.seed) ; np.random.seed(self.seed) ; random.seed(self.seed)
        self.episode_timestep = []
        self.episode_reward = []
        self.episode_seconds = []
        self.evaluation_scores = []
        self.episode_exploration = []
        global train_count
        train_count +=1
        self.updateNetwork()
        self.value_optimizer = self.optimizer
        self.replay_buffer = self.replay_buffer_fn()
        tempstate, _ = self.env.reset(seed=self.seed)
        self.explorationStrategyTrain(self.online_model, tempstate, init=True) #IMPORTANT

        max_episodes = self.MAX_TRAIN_EPISODES
        result = np.empty((2000, 5))
        result[:] = np.nan
        training_time = 0

        for episode in range(1, max_episodes + 1):
            episode_start = time.time()
            print(f'episode {episode} and train called {train_count} times ')
            state, _ = self.env.reset(seed=self.seed)
            self.episode_reward.append(0.0)
            self.episode_timestep.append(0.0)
            self.episode_exploration.append(0.0)

            for step in count():
                state, is_terminal = self.stepint(state, self.env)

                # Debug: Log Q-values
                with torch.no_grad():
                  current_Q_values = self.online_model(state).cpu().detach().numpy().squeeze()
                  #print(f"Debug: Episode {episode}, Step {step}, Q-values: {current_Q_values}")


                min_samples = self.replay_buffer.default_batch_size * self.batchSize
                if self.replay_buffer.length() > min_samples:
                    experiences = self.replay_buffer.sample()
                    experiences = self.online_model.load(experiences)
                    self.trainNetwork(experiences, self.epochs)

                if np.sum(self.episode_timestep) % self.updateFrequency == 0:
                    self.updateNetwork()

                if is_terminal:
                    break


            # Debug: Log replay buffer size
            #print(f"Episode: {episode}, Replay Buffer Size: {self.replay_buffer.length()}")

            episode_elapsed = time.time() - episode_start
            self.episode_seconds.append(episode_elapsed)
            training_time += episode_elapsed
            #print('first_evaluate_entered')
            evaluation_score= np.mean(self.evaluateAgent())
            #print('first_evaluate_exit')

            total_step = int(np.sum(self.episode_timestep))
            self.evaluation_scores.append(evaluation_score)

            mean_10_reward = np.mean(self.episode_reward[-10:])
            std_10_reward = np.std(self.episode_reward[-10:])
            mean_100_reward = np.mean(self.episode_reward[-100:])
            std_100_reward = np.std(self.episode_reward[-100:])
            mean_100_eval_score = np.mean(self.evaluation_scores[-100:])
            std_100_eval_score = np.std(self.evaluation_scores[-100:])
            #lst_100_exp_rat = np.array(self.episode_exploration[-100:])/np.array(self.episode_timestep[-100:])
            #mean_100_exp_rat = np.mean(lst_100_exp_rat)
            #std_100_exp_rat = np.std(lst_100_exp_rat)

            wallclock_elapsed = time.time() - training_start
            result[episode-1] = total_step, mean_100_reward, mean_100_eval_score, training_time, wallclock_elapsed
            reached_max_minutes = wallclock_elapsed >= 30 * 60
            reached_max_episodes = episode >= 10000
            reached_goal_mean_reward = mean_100_eval_score >= 475
            training_is_over = reached_max_minutes or reached_max_episodes or reached_goal_mean_reward
            if training_is_over:
                if reached_max_minutes: print(u'--> reached_max_minutes \u2715')
                if reached_max_episodes: print(u'--> reached_max_episodes \u2715')
                if reached_goal_mean_reward: print(u'--> reached_goal_mean_reward \u2713')
                break

        final_eval_rwd_list = self.evaluateAgent()
        mean_eval_rwd = np.mean(final_eval_rwd_list)
        wallclock_time = time.time() - training_start
        self.env.close()

        return result, training_time, final_eval_rwd_list, wallclock_time

In [ ]:
class DDQN(DDQN):
    def trainNetwork(self, experiences, epochs):
        # this method trains the value network epoch number of times and is called by the trainAgent function
        # it essentially uses the experiences to calculate target, using the targets it calculates the error, which
        # is further used for calulating the loss. It then uses the optimizer over the loss
        # to update the params of the network by backpropagating through the network
        # this function does not return anything
        # you can try out other loss functions other than MSE like Huber loss, MAE, etc.

        states, actions, rewards, next_states, is_terminals = experiences
        batch_size = len(is_terminals)

        # argmax_a_q_sp = self.target_model(next_states).max(1)[1]
        argmax_a_q_sp = self.online_model(next_states).max(1)[1]
        q_sp = self.target_model(next_states).detach()
        max_a_q_sp = q_sp[
            np.arange(batch_size), argmax_a_q_sp].unsqueeze(1)
        target_q_sa = rewards + (self.gamma * max_a_q_sp * (1 - is_terminals))
        q_sa = self.online_model(states).gather(1, actions)

        td_error = q_sa - target_q_sa
        value_loss = td_error.pow(2).mul(0.5).mean()
        self.value_optimizer.zero_grad()
        value_loss.backward()
        torch.nn.utils.clip_grad_norm_(self.online_model.parameters(),
                                       self.max_gradient_norm)
        self.value_optimizer.step()
        return

In [ ]:
class DDQN(DDQN):
    def updateNetwork(self):
        for target, online in zip(self.target_model.parameters(), self.online_model.parameters()):
            target.data.copy_(online.data)
        return

In [ ]:
class DDQN(DDQN):
    def evaluateAgent(self):
        rwd_list = []
        for eval_episode in range(self.MAX_EVAL_EPISODES):
            s, _ = self.env.reset(seed=self.seed)
            total_reward = 0
            rwd_list.append(0)
            for step in count():
                a = self.explorationStrategyEval(self.online_model, s)
                s, rwd, done, truncated,_ = self.env.step(a)
                total_reward += rwd

                # Debug: Log action, reward, and step
                #print(f"Evaluation Episode {eval_episode + 1}, Step {step}: State {s}, Action {a}, Reward {rwd}, Total Reward {total_reward}")

                rwd_list[-1] += rwd
                if done or truncated: break
        return rwd_list

In [ ]:
train_count=0
ddqn_results = []
best_agent, best_eval_score = None, float('-inf')
seed_list = [420, 133, 74, 317, 233]

for myseed in seed_list:

    val_optimizer_fn = lambda net, lr: optim.RMSprop(net.parameters(), lr=lr)

    # Instantiation of the DQN class
    ddqn_instance = DDQN(
        env_id='CartPole-v1',
        seed=myseed,
        gamma=0.99,
        epochs = 20,
        bufferSize=50000,
        batchSize=5,
        optimizerFn=val_optimizer_fn,
        optimizerLR=1e-3,
        max_gradient_norm = float('inf'),
        MAX_TRAIN_EPISODES=500,
        MAX_EVAL_EPISODES=1,
        explorationStrategyTrainFn=selectEpsilonExpGreedyAction,
        explorationStrategyEvalFn=selectGreedyAction,
        updateFrequency=10
    )

    # Running the NFQ method and appending results
    trainRewardsList, trainTimeList, evalRewardsList, wallClockTimeList, final_eval_score = ddqn_instance.runDDQN()
    ddqn_results.append(trainRewardsList)
    if final_eval_score > best_eval_score:
        best_eval_score = final_eval_score
        best_agent = ddqn_instance

# Convert dqn_results to a numpy array for any further processing
ddqn_results = np.array(ddqn_results)

In [ ]:
reward_max, steps_max, eval_score_max, train_time_max, wall_clock_max = np.max(ddqn_results, axis=0).T
reward_min, steps_min, eval_score_min, train_time_min, wall_clock_min = np.min(ddqn_results, axis=0).T
reward_avg, steps_avg, eval_score_avg, train_time_avg, wall_clock_avg = np.mean(ddqn_results, axis=0).T
episode_indices = np.arange(len(reward_avg))

plt.style.use('ggplot')  
fig, plot_areas = plt.subplots(5, 1, figsize=(12, 25), sharex='col')
fig.subplots_adjust(hspace=0.5)  
colors = ['blue', 'green', 'red', 'purple', 'orange']
titles = ['Total Steps','Training Reward', 'Evaluation reward',  'Training Duration', 'Wall-clock Time']
y_labels = ['Steps', 'Reward', 'Reward', 'Seconds', 'Seconds']
data_max = [reward_max, steps_max, eval_score_max, train_time_max, wall_clock_max]
data_min = [reward_min, steps_min, eval_score_min, train_time_min, wall_clock_min]
data_avg = [reward_avg, steps_avg, eval_score_avg, train_time_avg, wall_clock_avg]

# Generate plots
for ax, title, color, max_data, min_data, avg_data, y_label in zip(plot_areas, titles, colors, data_max, data_min, data_avg, y_labels):
    ax.plot(max_data, linestyle='--', color=color, alpha=0.75)
    ax.plot(min_data, linestyle='--', color=color, alpha=0.75)
    ax.plot(avg_data, label='Average', color=color, linewidth=2)
    ax.fill_between(episode_indices, min_data, max_data, color=color, alpha=0.2)
    ax.set_title(title)
    ax.set_ylabel(y_label)
    ax.legend()

plot_areas[-1].set_xlabel('Episodes')
plt.show()


## Dueling DDQN

Implement the Dueling Double Deep Q algorithm. We have studied about Dueling Double DQN agent in the Lecture. Use the function definitions (given below).

This class implements the Dueling Double DQN agent, you are required to implement the various methods of this class
as outlined below. Note this class is generic and should work with any permissible Gym environment

```
class D3QN():
    def __init__(env, seed, gamma, tau,
                 bufferSize,
                 batchSize,
                 optimizerFn,
                 optimizerLR,
                 MAX_TRAIN_EPISODES, MAX_EVAL_EPISODES,
                 explorationStrategyTrainFn,
                 explorationStrategyEvalFn,
                 updateFrequency)
    def initBookKeeping(self)
    def performBookKeeping(self, train = True)
    def runD3QN(self)
    def trainAgent(self)
    def trainNetwork(self, experiences)
    def updateNetwork(self, onlineNet, targetNet)
    def evaluateAgent(self)
```

In [ ]:
class D3QN():
    def __init__(self,env_id, seed, gamma,epochs,
                 bufferSize,
                 batchSize,
                 optimizerFn,
                 optimizerLR,
                 max_gradient_norm,
                 MAX_TRAIN_EPISODES, MAX_EVAL_EPISODES,
                 explorationStrategyTrainFn,
                 explorationStrategyEvalFn,
                 updateFrequency, tau):
        
        #this D3QN method
        # 1. creates and initializes (with seed) the environment, train/eval episodes, gamma, etc.
        # 2. creates and intializes all the variables required for book-keeping values via the initBookKeeping method
        # 3. creates tareget and online Q-networks using the createValueNetwork above
        # 4. creates and initializes (with network params) the optimizer function
        # 5. sets the explorationStartegy variables/functions for train and evaluation
        # 6. sets the batchSize for the number of experiences
        # 7. Creates the replayBuffer

        #Your code goes in here

        # 1. Environment initialization using env_id

        self.env = gym.make(env_id)
        self.seed = seed
        reset_env(self.env, seed=seed)  # Seed the environment (affects environment operations)

        # Note on seeding:
        # Seeding affects global state for NumPy and PyTorch, which might impact other parts of an application.
        # It's set here for reproducibility within the scope of this NFQ instance's operations.
        torch.manual_seed(seed)  # Seed PyTorch (global effect)
        np.random.seed(seed)  # Seed NumPy (global effect)
        random.seed(seed)

        # 2. Book-keeping initialization
        #self.initBookKeeping()

        # 3. Creating online Q-network
        inDim = self.env.observation_space.shape[0]
        outDim = self.env.action_space.n
        hDim = [512,128]  # Example hidden layer dimensions, adjust as needed
        activation = torch.nn.functional.relu  # Specify the activation function
        self.target_model = createDuelingNetwork(inDim, outDim, hDim, activation)
        self.online_model = createDuelingNetwork(inDim, outDim, hDim, activation)

        # 4. Optimizer initialization
        self.optimizer = optimizerFn(self.online_model, optimizerLR)

        # 5. Exploration strategy setup
        self.explorationStrategyTrain = explorationStrategyTrainFn
        self.explorationStrategyEval = explorationStrategyEvalFn

        # 6. Other variables
        self.gamma = gamma
        self.epochs = epochs
        self.batchSize = batchSize
        self.MAX_TRAIN_EPISODES = MAX_TRAIN_EPISODES
        self.MAX_EVAL_EPISODES = MAX_EVAL_EPISODES
        self.updateFrequency = updateFrequency
        self.max_gradient_norm = max_gradient_norm
        self.tau = tau

        # 7. Replay buffer
        self.replay_buffer_fn = lambda: ReplayBuffer(bufferSize=50000)

In [ ]:
class D3QN(D3QN):
    def initBookKeeping(self):
        #this method creates and intializes all the variables required for book-keeping values and it is called
        #init method
        #
        # Your code goes in here
        #

        return

In [ ]:
class D3QN(D3QN):
    def performBookKeeping(self, train = True):
        #this method updates relevant variables for the bookKeeping, this can be called
        #multiple times during training
        #if you want you can print information using this, so it may help to monitor progress and also help to debug
        #
        # Your code goes in here
        #

        return

In [ ]:
class D3QN(D3QN):
    def stepint(self, state, env):

        action = self.explorationStrategyTrain(self.online_model, state)
        new_state, reward, is_terminal, is_truncated, info = env.step(action)
        is_failure = is_terminal and not is_truncated
        experience = (state, action, reward, new_state, float(is_failure))
        self.replay_buffer.store(experience)
        self.episode_reward[-1] += reward
        self.episode_timestep[-1] += 1
        #self.episode_exploration[-1] += int(self.training_strategy.exploratory_action_taken)
        return new_state, is_terminal

In [ ]:
class D3QN(D3QN):
    def runD3QN(self):
        #this is the main method, it trains the agent, performs bookkeeping while training and finally evaluates
        #the agent and returns the following quantities:
        #1. episode wise mean train rewards
        #2. epsiode wise mean eval rewards
        #2. episode wise trainTime (in seconds): time elapsed during training since the start of the first episode
        #3. episode wise wallClockTime (in seconds): actual time elapsed since the start of training,
        #                               note this will include time for BookKeeping and evaluation
        # Note both trainTime and wallClockTime get accumulated as episodes proceed.

        #Your code goes in here
        resultList, trainTimeList, evalRewardsList, wallClockTimeList = self.trainAgent()
        resultEval = self.evaluateAgent()
        finalEvalReward  = np.mean(resultEval)

        return resultList, trainTimeList, evalRewardsList, wallClockTimeList, finalEvalReward


In [ ]:
class D3QN(D3QN):
    def trainAgent(self):
        #this method collects experiences and trains the agent and does BookKeeping while training.
        #this calls the trainNetwork() method internally, it also evaluates the agent per episode
        #it trains the agent for MAX_TRAIN_EPISODES
        #
        training_start = time.time()
        torch.manual_seed(self.seed) ; np.random.seed(self.seed) ; random.seed(self.seed)
        self.episode_timestep = []
        self.episode_reward = []
        self.episode_seconds = []
        self.evaluation_scores = []
        self.episode_exploration = []
        global train_count
        train_count +=1
        self.updateNetwork(tau=1.0)
        self.value_optimizer = self.optimizer
        self.replay_buffer = self.replay_buffer_fn()
        tempstate, _ = self.env.reset(seed=self.seed)
        self.explorationStrategyTrain(self.online_model, tempstate, init=True) #IMPORTANT

        max_episodes = self.MAX_TRAIN_EPISODES
        result = np.empty((2000, 5))
        result[:] = np.nan
        training_time = 0

        for episode in range(1, max_episodes + 1):
            episode_start = time.time()
            print(f'episode {episode} and train called {train_count} times ')
            state, _ = self.env.reset(seed=self.seed)
            self.episode_reward.append(0.0)
            self.episode_timestep.append(0.0)
            self.episode_exploration.append(0.0)

            for step in count():
                state, is_terminal = self.stepint(state, self.env)

                # Debug: Log Q-values
                with torch.no_grad():
                  current_Q_values = self.online_model(state).cpu().detach().numpy().squeeze()
                  #print(f"Debug: Episode {episode}, Step {step}, Q-values: {current_Q_values}")


                min_samples = self.replay_buffer.default_batch_size * self.batchSize
                if self.replay_buffer.length() > min_samples:
                    experiences = self.replay_buffer.sample()
                    experiences = self.online_model.load(experiences)
                    self.trainNetwork(experiences, self.epochs)

                if np.sum(self.episode_timestep) % self.updateFrequency == 0:
                    self.updateNetwork()

                if is_terminal:
                    break


            # Debug: Log replay buffer size
            #print(f"Episode: {episode}, Replay Buffer Size: {self.replay_buffer.length()}")

            episode_elapsed = time.time() - episode_start
            self.episode_seconds.append(episode_elapsed)
            training_time += episode_elapsed
            #print('first_evaluate_entered')
            evaluation_score= np.mean(self.evaluateAgent())
            #print('first_evaluate_exit')

            total_step = int(np.sum(self.episode_timestep))
            self.evaluation_scores.append(evaluation_score)

            mean_10_reward = np.mean(self.episode_reward[-10:])
            std_10_reward = np.std(self.episode_reward[-10:])
            mean_100_reward = np.mean(self.episode_reward[-100:])
            std_100_reward = np.std(self.episode_reward[-100:])
            mean_100_eval_score = np.mean(self.evaluation_scores[-100:])
            std_100_eval_score = np.std(self.evaluation_scores[-100:])
            #lst_100_exp_rat = np.array(self.episode_exploration[-100:])/np.array(self.episode_timestep[-100:])
            #mean_100_exp_rat = np.mean(lst_100_exp_rat)
            #std_100_exp_rat = np.std(lst_100_exp_rat)

            wallclock_elapsed = time.time() - training_start
            result[episode-1] = total_step, mean_100_reward, mean_100_eval_score, training_time, wallclock_elapsed
            reached_max_minutes = wallclock_elapsed >= 30 * 60
            reached_max_episodes = episode >= 10000
            reached_goal_mean_reward = mean_100_eval_score >= 475
            training_is_over = reached_max_minutes or reached_max_episodes or reached_goal_mean_reward
            if training_is_over:
                if reached_max_minutes: print(u'--> reached_max_minutes \u2715')
                if reached_max_episodes: print(u'--> reached_max_episodes \u2715')
                if reached_goal_mean_reward: print(u'--> reached_goal_mean_reward \u2713')
                break

        final_eval_rwd_list = self.evaluateAgent()
        mean_eval_rwd = np.mean(final_eval_rwd_list)
        wallclock_time = time.time() - training_start
        self.env.close()

        return result, training_time, final_eval_rwd_list, wallclock_time

In [ ]:
class D3QN(D3QN):
    def trainNetwork(self, experiences, epochs):
        # this method trains the value network epoch number of times and is called by the trainAgent function
        # it essentially uses the experiences to calculate target, using the targets it calculates the error, which
        # is further used for calulating the loss. It then uses the optimizer over the loss
        # to update the params of the network by backpropagating through the network
        # this function does not return anything
        # you can try out other loss functions other than MSE like Huber loss, MAE, etc.

        #Your code goes in here
        
        states, actions, rewards, next_states, is_terminals = experiences
        batch_size = len(is_terminals)

        argmax_a_q_sp = self.online_model(next_states).max(1)[1]
        q_sp = self.target_model(next_states).detach()
        max_a_q_sp = q_sp[
            np.arange(batch_size), argmax_a_q_sp].unsqueeze(1)
        target_q_sa = rewards + (self.gamma * max_a_q_sp * (1 - is_terminals))
        q_sa = self.online_model(states).gather(1, actions)

        td_error = q_sa - target_q_sa
        value_loss = td_error.pow(2).mul(0.5).mean()
        self.value_optimizer.zero_grad()
        value_loss.backward()        
        torch.nn.utils.clip_grad_norm_(self.online_model.parameters(), 
                                       self.max_gradient_norm)
        self.value_optimizer.step()
        return

In [ ]:
class D3QN(D3QN):
    def updateNetwork(self, tau=None):
        #this function updates the onlineNetwork with the target network using Polyak averaging
        #
        # Your code goes in here
        tau = self.tau if tau is None else tau
        for target, online in zip(self.target_model.parameters(), 
                                  self.online_model.parameters()):
            target_ratio = (1.0 - tau) * target.data
            online_ratio = tau * online.data
            mixed_weights = target_ratio + online_ratio
            target.data.copy_(mixed_weights)
            
        return

In [ ]:
class D3QN(D3QN):
    def evaluateAgent(self):
        rwd_list = []
        for eval_episode in range(self.MAX_EVAL_EPISODES):
            s, _ = self.env.reset(seed=self.seed)
            total_reward = 0
            rwd_list.append(0)
            for step in count():
                a = self.explorationStrategyEval(self.online_model, s)
                s, rwd, done, truncated,_ = self.env.step(a)
                total_reward += rwd

                rwd_list[-1] += rwd
                if done or truncated: break
        return rwd_list

In [ ]:
train_count=0
d3qn_results = []
best_agent, best_eval_score = None, float('-inf')
seed_list = [420, 133, 74, 317, 233]

for myseed in seed_list:

    val_optimizer_fn = lambda net, lr: optim.RMSprop(net.parameters(), lr=lr)

    # Instantiation of the D3QN class
    d3qn_instance = D3QN(
        env_id='CartPole-v1',
        seed=myseed,
        gamma=0.99,
        epochs = 40,
        bufferSize=50000,
        batchSize=5,
        optimizerFn=val_optimizer_fn,
        optimizerLR=0.0005,
        max_gradient_norm = float('inf'),
        MAX_TRAIN_EPISODES=300,
        MAX_EVAL_EPISODES=1,
        explorationStrategyTrainFn=selectEpsilonExpGreedyAction,
        explorationStrategyEvalFn=selectGreedyAction,
        updateFrequency=1,
        tau = 0.1
    )

    # Running the NFQ method and appending results
    trainRewardsList, trainTimeList, evalRewardsList, wallClockTimeList, final_eval_score = d3qn_instance.runD3QN()
    d3qn_results.append(trainRewardsList)
    if final_eval_score > best_eval_score:
        best_eval_score = final_eval_score
        best_agent = d3qn_instance

# Convert dqn_results to a numpy array for any further processing
d3qn_results = np.array(d3qn_results)

In [ ]:
reward_max, steps_max, eval_score_max, train_time_max, wall_clock_max = np.max(d3qn_results, axis=0).T
reward_min, steps_min, eval_score_min, train_time_min, wall_clock_min = np.min(d3qn_results, axis=0).T
reward_avg, steps_avg, eval_score_avg, train_time_avg, wall_clock_avg = np.mean(d3qn_results, axis=0).T
episode_indices = np.arange(len(reward_avg))

plt.style.use('ggplot')  
fig, plot_areas = plt.subplots(5, 1, figsize=(12, 25), sharex='col')
fig.subplots_adjust(hspace=0.5)  
colors = ['blue', 'green', 'red', 'purple', 'orange']
titles = ['Total Steps','Training Reward', 'Evaluation reward',  'Training Duration', 'Wall-clock Time']
y_labels = ['Steps', 'Reward', 'Reward', 'Seconds', 'Seconds']
data_max = [reward_max, steps_max, eval_score_max, train_time_max, wall_clock_max]
data_min = [reward_min, steps_min, eval_score_min, train_time_min, wall_clock_min]
data_avg = [reward_avg, steps_avg, eval_score_avg, train_time_avg, wall_clock_avg]

# Generate plots
for ax, title, color, max_data, min_data, avg_data, y_label in zip(plot_areas, titles, colors, data_max, data_min, data_avg, y_labels):
    ax.plot(max_data, linestyle='--', color=color, alpha=0.75)
    ax.plot(min_data, linestyle='--', color=color, alpha=0.75)
    ax.plot(avg_data, label='D3QN', color=color, linewidth=2)
    ax.fill_between(episode_indices, min_data, max_data, color=color, alpha=0.2)
    ax.set_title(title)
    ax.set_ylabel(y_label)
    ax.legend()

plot_areas[-1].set_xlabel('Episodes')
plt.show()

## Dueling Double Deep Q Network with Prioritized Experience Replay (D3QN-PER)

Implement the Dueling Double DQN with Prioritized Experience Replay (D3QN-PER) agent. We have studied about D3QN-PER agent in the Lecture. Use the function definitions (given below).

This class implements the D3QN-PER agent, you are required to implement the various methods of this class
as outlined below. Note this class is generic and should work with any permissible Gym environment

```
class D3QN_PER():
    def __init__(env, seed, gamma, tau, alpha, beta, beta_rate,
                 bufferSize,
                 batchSize,
                 optimizerFn,
                 optimizerLR,
                 MAX_TRAIN_EPISODES, MAX_EVAL_EPISODES,
                 explorationStrategyTrainFn,
                 explorationStrategyEvalFn,
                 updateFrequency)
    def initBookKeeping(self)
    def performBookKeeping(self, train = True)
    def runD3QN_PER(self)
    def trainAgent(self)
    def trainNetwork(self, experiences)
    def updateNetwork(self, onlineNet, targetNet)
    def evaluateAgent(self)
```

In [ ]:
class D3QN_PER():
    def __init__(self,env_id, seed, gamma,epochs,
                 bufferSize,
                 batchSize,
                 optimizerFn,
                 optimizerLR,
                 max_gradient_norm,
                 MAX_TRAIN_EPISODES, MAX_EVAL_EPISODES,
                 explorationStrategyTrainFn,
                 explorationStrategyEvalFn,
                 updateFrequency, tau):
        
        #thisD3QN_PER method
        # 1. creates and initializes (with seed) the environment, train/eval episodes, gamma, etc.
        # 2. creates and intializes all the variables required for book-keeping values via the initBookKeeping method
        # 3. creates tareget and online Q-networks using the createValueNetwork above
        # 4. creates and initializes (with network params) the optimizer function
        # 5. sets the explorationStartegy variables/functions for train and evaluation
        # 6. sets the batchSize for the number of experiences
        # 7. Creates the replayBuffer

        #Your code goes in here

        # 1. Environment initialization using env_id

        self.env = gym.make(env_id)
        self.seed = seed
        reset_env(self.env, seed=seed)  # Seed the environment (affects environment operations)

        # Note on seeding:
        # Seeding affects global state for NumPy and PyTorch, which might impact other parts of an application.
        # It's set here for reproducibility within the scope of this NFQ instance's operations.
        torch.manual_seed(seed)  # Seed PyTorch (global effect)
        np.random.seed(seed)  # Seed NumPy (global effect)
        random.seed(seed)

        # 2. Book-keeping initialization
        #self.initBookKeeping()

        # 3. Creating online Q-network
        inDim = self.env.observation_space.shape[0]
        outDim = self.env.action_space.n
        hDim = [512,128]  # Example hidden layer dimensions, adjust as needed
        activation = torch.nn.functional.relu  # Specify the activation function
        self.target_model = createDuelingNetwork(inDim, outDim, hDim, activation)
        self.online_model = createDuelingNetwork(inDim, outDim, hDim, activation)

        # 4. Optimizer initialization
        self.optimizer = optimizerFn(self.online_model, optimizerLR)

        # 5. Exploration strategy setup
        self.explorationStrategyTrain = explorationStrategyTrainFn
        self.explorationStrategyEval = explorationStrategyEvalFn

        # 6. Other variables
        self.gamma = gamma
        self.epochs = epochs
        self.batchSize = batchSize
        self.MAX_TRAIN_EPISODES = MAX_TRAIN_EPISODES
        self.MAX_EVAL_EPISODES = MAX_EVAL_EPISODES
        self.updateFrequency = updateFrequency
        self.max_gradient_norm = max_gradient_norm
        self.tau = tau

        # 7. Replay buffer
        self.replay_buffer_fn = lambda: ReplayBuffer(bufferSize=20000, bufferType = "D3QN-PER", max_samples=bufferSize, rank_based=True,alpha=0.6, beta=0.1, beta_rate=0.9999)

In [ ]:
class D3QN_PER(D3QN_PER):
    def initBookKeeping(self):
        #this method creates and intializes all the variables required for book-keeping values and it is called
        #init method
        #
        #Your code goes in here
        #

        return

In [ ]:
class D3QN_PER(D3QN_PER):
    def performBookKeeping(self, train = True):
        #this method updates relevant variables for the bookKeeping, this can be called
        #multiple times during training
        #if you want you can print information using this, so it may help to monitor progress and also help to debug
        #
        #Your code goes in here
        #

        return

In [ ]:
class D3QN_PER(D3QN_PER):
    
    def stepint(self, state, env):

        action = self.explorationStrategyTrain(self.online_model, state)
        new_state, reward, is_terminal, is_truncated, info = env.step(action)
        is_failure = is_terminal and not is_truncated
        experience = (state, action, reward, new_state, float(is_failure))
        self.replay_buffer.store(experience)
        self.episode_reward[-1] += reward
        self.episode_timestep[-1] += 1
        #self.episode_exploration[-1] += int(self.training_strategy.exploratory_action_taken)
        return new_state, is_terminal

In [ ]:
class D3QN_PER(D3QN_PER):
    def runD3QN_PER(self):
        #this is the main method, it trains the agent, performs bookkeeping while training and finally evaluates
        #the agent and returns the following quantities:
        #1. episode wise mean train rewards
        #2. epsiode wise mean eval rewards
        #2. episode wise trainTime (in seconds): time elapsed during training since the start of the first episode
        #3. episode wise wallClockTime (in seconds): actual time elapsed since the start of training,
        #                               note this will include time for BookKeeping and evaluation
        # Note both trainTime and wallClockTime get accumulated as episodes proceed.

        resultList, trainTimeList, evalRewardsList, wallClockTimeList = self.trainAgent()
        resultEval = self.evaluateAgent()
        finalEvalReward  = np.mean(resultEval)

        return resultList, trainTimeList, evalRewardsList, wallClockTimeList, finalEvalReward

In [ ]:
class D3QN_PER(D3QN_PER):

    def trainAgent(self):
        #this method collects experiences and trains the agent and does BookKeeping while training.
        #this calls the trainNetwork() method internally, it also evaluates the agent per episode
        #it trains the agent for MAX_TRAIN_EPISODES
        #
        training_start = time.time()
        torch.manual_seed(self.seed) ; np.random.seed(self.seed) ; random.seed(self.seed)
        self.episode_timestep = []
        self.episode_reward = []
        self.episode_seconds = []
        self.evaluation_scores = []
        self.episode_exploration = []
        global train_count
        train_count +=1
        self.updateNetwork(tau=1.0)
        self.value_optimizer = self.optimizer
        self.replay_buffer = self.replay_buffer_fn()
        tempstate, _ = self.env.reset(seed=self.seed)
        self.explorationStrategyTrain(self.online_model, tempstate, init=True) #IMPORTANT

        max_episodes = self.MAX_TRAIN_EPISODES
        result = np.empty((2000, 5))
        result[:] = np.nan
        training_time = 0

        for episode in range(1, max_episodes + 1):
            episode_start = time.time()
            print(f'episode {episode} and train called {train_count} times ')
            state, _ = self.env.reset(seed=self.seed)
            self.episode_reward.append(0.0)
            self.episode_timestep.append(0.0)
            self.episode_exploration.append(0.0)

            for step in count():
                state, is_terminal = self.stepint(state, self.env)

                # Debug: Log Q-values
                with torch.no_grad():
                  current_Q_values = self.online_model(state).cpu().detach().numpy().squeeze()
                  #print(f"Debug: Episode {episode}, Step {step}, Q-values: {current_Q_values}")


                min_samples = self.replay_buffer.batch_size * self.batchSize
                if self.replay_buffer.length() > min_samples:
                    idxs, weights, samples = self.replay_buffer.sample()
                    experiences = self.online_model.load(samples)
                    experiences = (idxs, weights) + (experiences,)
                    self.trainNetwork(experiences)

                if np.sum(self.episode_timestep) % self.updateFrequency == 0:
                    self.updateNetwork()

                if is_terminal:
                    break


            # Debug: Log replay buffer size
            #print(f"Episode: {episode}, Replay Buffer Size: {self.replay_buffer.length()}")

            episode_elapsed = time.time() - episode_start
            self.episode_seconds.append(episode_elapsed)
            training_time += episode_elapsed
            #print('first_evaluate_entered')
            evaluation_score= np.mean(self.evaluateAgent())
            #print('first_evaluate_exit')

            total_step = int(np.sum(self.episode_timestep))
            self.evaluation_scores.append(evaluation_score)

            mean_10_reward = np.mean(self.episode_reward[-10:])
            std_10_reward = np.std(self.episode_reward[-10:])
            mean_100_reward = np.mean(self.episode_reward[-100:])
            std_100_reward = np.std(self.episode_reward[-100:])
            mean_100_eval_score = np.mean(self.evaluation_scores[-100:])
            std_100_eval_score = np.std(self.evaluation_scores[-100:])
            #lst_100_exp_rat = np.array(self.episode_exploration[-100:])/np.array(self.episode_timestep[-100:])
            #mean_100_exp_rat = np.mean(lst_100_exp_rat)
            #std_100_exp_rat = np.std(lst_100_exp_rat)

            wallclock_elapsed = time.time() - training_start
            result[episode-1] = total_step, mean_100_reward, mean_100_eval_score, training_time, wallclock_elapsed
            reached_max_minutes = wallclock_elapsed >= 30 * 60
            reached_max_episodes = episode >= 10000
            reached_goal_mean_reward = mean_100_eval_score >= 475
            training_is_over = reached_max_minutes or reached_max_episodes or reached_goal_mean_reward
            if training_is_over:
                if reached_max_minutes: print(u'--> reached_max_minutes \u2715')
                if reached_max_episodes: print(u'--> reached_max_episodes \u2715')
                if reached_goal_mean_reward: print(u'--> reached_goal_mean_reward \u2713')
                break

        final_eval_rwd_list = self.evaluateAgent()
        mean_eval_rwd = np.mean(final_eval_rwd_list)
        wallclock_time = time.time() - training_start
        self.env.close()

        return result, training_time, final_eval_rwd_list, wallclock_time

In [ ]:
class D3QN_PER(D3QN_PER):
    def trainNetwork(self, experiences):
        # this method trains the value network epoch number of times and is called by the trainAgent function
        # it essentially uses the experiences to calculate target, using the targets it calculates the error, which
        # is further used for calulating the loss. It then uses the optimizer over the loss
        # to update the params of the network by backpropagating through the network
        # this function does not return anything
        # you can try out other loss functions other than MSE like Huber loss, MAE, etc.
        #
        #Your code goes in here
        idxs, weights, (states, actions, rewards, next_states, is_terminals) = experiences
        weights = self.online_model.numpy_float_to_device(weights)
        batch_size = len(is_terminals)
        
        argmax_a_q_sp = self.online_model(next_states).max(1)[1]
        q_sp = self.target_model(next_states).detach()
        max_a_q_sp = q_sp[np.arange(batch_size), argmax_a_q_sp].unsqueeze(1)
        target_q_sa = rewards + (self.gamma * max_a_q_sp * (1 - is_terminals))
        q_sa = self.online_model(states).gather(1, actions)

        td_error = q_sa - target_q_sa
        value_loss = (weights * td_error).pow(2).mul(0.5).mean()
        self.value_optimizer.zero_grad()
        value_loss.backward()        
        torch.nn.utils.clip_grad_norm_(self.online_model.parameters(), 
                                       self.max_gradient_norm)
        self.value_optimizer.step()

        priorities = np.abs(td_error.detach().cpu().numpy())
        self.replay_buffer.update(idxs, priorities)

        return

In [ ]:
class D3QN_PER(D3QN_PER):
    def updateNetwork(self, tau=None):
        #this function updates the onlineNetwork with the target network using Polyak averaging 
        tau = self.tau if tau is None else tau
        for target, online in zip(self.target_model.parameters(), 
                                  self.online_model.parameters()):
            target_ratio = (1.0 - tau) * target.data
            online_ratio = tau * online.data
            mixed_weights = target_ratio + online_ratio
            target.data.copy_(mixed_weights)
 

In [ ]:
class D3QN_PER(D3QN_PER):
    def evaluateAgent(self):
        #this function evaluates the agent using the value network, it evaluates agent for MAX_EVAL_EPISODES
        #typcially MAX_EVAL_EPISODES = 1
        #
        #Your code goes in here
        rwd_list = []
        for eval_episode in range(self.MAX_EVAL_EPISODES):
            s, _ = self.env.reset(seed=self.seed)
            total_reward = 0
            rwd_list.append(0)
            for step in count():
                a = self.explorationStrategyEval(self.online_model, s)
                s, rwd, done, truncated,_ = self.env.step(a)
                total_reward += rwd

                rwd_list[-1] += rwd
                if done or truncated: break
        return rwd_list


In [ ]:
train_count=0
d3qnper_results = []
best_agent, best_eval_score = None, float('-inf')
seed_list = [420, 133, 74, 317, 233]

for myseed in seed_list:

    val_optimizer_fn = lambda net, lr: optim.RMSprop(net.parameters(), lr=lr)

    # Instantiation of the D3QN_PER class
    d3qnper_instance = D3QN_PER(
        env_id='CartPole-v1',
        seed=myseed,
        gamma=0.99,
        epochs = 20,
        bufferSize=20000,
        batchSize=5,
        optimizerFn=val_optimizer_fn,
        optimizerLR=0.0005,
        max_gradient_norm = float('inf'),
        MAX_TRAIN_EPISODES=300,
        MAX_EVAL_EPISODES=1,
        explorationStrategyTrainFn=selectEpsilonExpGreedyAction,
        explorationStrategyEvalFn=selectGreedyAction,
        updateFrequency=1,
        tau = 0.01
    )

    # Running the NFQ method and appending results
    trainRewardsList, trainTimeList, evalRewardsList, wallClockTimeList, final_eval_score = d3qnper_instance.runD3QN_PER()
    d3qnper_results.append(trainRewardsList)
    if final_eval_score > best_eval_score:
        best_eval_score = final_eval_score
        best_agent = d3qnper_instance

# Convert dqn_results to a numpy array for any further processing
d3qnper_results = np.array(d3qnper_results)

In [ ]:
reward_max, steps_max, eval_score_max, train_time_max, wall_clock_max = np.max(d3qnper_results, axis=0).T
reward_min, steps_min, eval_score_min, train_time_min, wall_clock_min = np.min(d3qnper_results, axis=0).T
reward_avg, steps_avg, eval_score_avg, train_time_avg, wall_clock_avg = np.mean(d3qnper_results, axis=0).T
episode_indices = np.arange(len(reward_avg))

plt.style.use('ggplot')  
fig, plot_areas = plt.subplots(5, 1, figsize=(12, 25), sharex='col')
fig.subplots_adjust(hspace=0.5)  
colors = ['blue', 'green', 'red', 'purple', 'orange']
titles = ['Total Steps','Training Reward', 'Evaluation reward',  'Training Duration', 'Wall-clock Time']
y_labels = ['Steps', 'Reward', 'Reward', 'Seconds', 'Seconds']
data_max = [reward_max, steps_max, eval_score_max, train_time_max, wall_clock_max]
data_min = [reward_min, steps_min, eval_score_min, train_time_min, wall_clock_min]
data_avg = [reward_avg, steps_avg, eval_score_avg, train_time_avg, wall_clock_avg]

# Generate plots
for ax, title, color, max_data, min_data, avg_data, y_label in zip(plot_areas, titles, colors, data_max, data_min, data_avg, y_labels):
    ax.plot(max_data, linestyle='--', color=color, alpha=0.75)
    ax.plot(min_data, linestyle='--', color=color, alpha=0.75)
    ax.plot(avg_data, label='D3QN-PER', color=color, linewidth=2)
    ax.fill_between(episode_indices, min_data, max_data, color=color, alpha=0.2)
    ax.set_title(title)
    ax.set_ylabel(y_label)
    ax.legend()

plot_areas[-1].set_xlabel('Episodes')
plt.show()

# Deep Policy Based RL agents.
<a id="deep-policy-based"></a>

### The purpose of this part is to learn about different Deep Policy Based RL agents.

In this part of the assignment you will be implementing Deep Policy based RL algorithms we learnt in Lectures. Namely, we will be implementing REINFORCE and VPG.

For all the algorithms below, this time we will not be specifying the hyper-parameters, please play with the hyper-params to come up with the best values. This way you will learn to tune the model. Some of the values were specified in the lecture, that would be a good starting point.

## REINFORCE

Implement the REINFORCE algorithm. We have studied about REINFORCE algorithm in the Lecture. Use the function definitions (given below).

This class implements the REINFORCE Agent, you are required to implement the various methods of this class
as outlined below. Note this class is generic and should work with any permissible Gym environment

```
class REINFORCE():
    def __init__(env, seed, gamma,
                 optimizerFn,
                 optimizerLR,
                 MAX_TRAIN_EPISODES, MAX_EVAL_EPISODES,
                 explorationStrategyTrainFn,
                 explorationStrategyEvalFn)
    def initBookKeeping(self)
    def performBookKeeping(self, train = True)
    def runREINFORCE(self)
    def trainAgent(self)
    def trainPolicyNetwork(self, experiences)
    def evaluateAgent(self)
```

### Implement the methods for the REINFORCE class below

In [ ]:
class REINFORCE():

    def __init__(self, env_id, seed, gamma, optimizerFn, optimizerLR, MAX_TRAIN_EPISODES, MAX_EVAL_EPISODES):

        # 1. Environment initialization using env_id
        self.env = gym.make(env_id)
        self.seed = seed
        reset_env(self.env, seed=seed)  # Seed the environment
        torch.manual_seed(seed)  # Seed PyTorch
        np.random.seed(seed)  # Seed NumPy
        random.seed(seed)

        # 3. Creating P-network
        inDim = self.env.observation_space.shape[0]
        outDim = self.env.action_space.n
        hDim = [128, 64]  # Hidden layer dimensions
        activation = torch.nn.functional.relu
        self.p_network = createPolicyNetworkREINFORCE(inDim, outDim, hDim, activation)

        # 4. Optimizer initialization
        self.optimizer = optimizerFn(self.p_network.parameters(), optimizerLR)

        # 6. Other variables
        self.gamma = gamma
        self.MAX_TRAIN_EPISODES = MAX_TRAIN_EPISODES
        self.MAX_EVAL_EPISODES = MAX_EVAL_EPISODES

    def runREINFORCE(self):
        resultList, trainTimeList, evalRewardsList, wallClockTimeList = self.trainAgent()
        resultEval = self.evaluateAgent()
        finalEvalReward = np.mean(resultEval)

        return resultList, trainTimeList, evalRewardsList, wallClockTimeList, finalEvalReward

    def stepint(self, state, env):
        action, is_exploratory, logpa, _ = self.policy_model.full_pass(state)
        new_state, reward, is_terminal, _, _ = env.step(action)

        self.logpas.append(logpa)
        self.rewards.append(reward)
        self.episode_reward[-1] += reward
        self.episode_timestep[-1] += 1
        self.episode_exploration[-1] += int(is_exploratory)

        return new_state, is_terminal

    def trainAgent(self):
        training_start = time.time()
        torch.manual_seed(self.seed) ; np.random.seed(self.seed) ; random.seed(self.seed)


        self.episode_timestep = []
        self.episode_reward = []
        self.episode_seconds = []
        self.episode_exploration = []
        self.evaluation_scores = []
        global train_count
        global trunc
        trunc  = 0
        train_count +=1

        self.policy_model = self.p_network
        max_episodes = self.MAX_TRAIN_EPISODES
        self.policy_optimizer = self.optimizer

        result = np.empty((max_episodes, 5))
        result[:] = np.nan
        training_time = 0
        for episode in range(1, max_episodes + 1):
            episode_start = time.time()
            print(f'episode {episode} and train count is {train_count}')
            state, _ = self.env.reset(seed=self.seed)
            self.episode_reward.append(0.0)
            self.episode_timestep.append(0.0)
            self.episode_exploration.append(0.0)

            # collect rollout
            self.logpas, self.rewards = [], []
            for step in count():
                state, is_terminal = self.stepint(state,self.env)
                if is_terminal:
                    break

            self.trainNetwork()

            episode_elapsed = time.time() - episode_start
            self.episode_seconds.append(episode_elapsed)
            training_time += episode_elapsed
            evaluation_score= np.mean(self.evaluateAgent())

            total_step = int(np.sum(self.episode_timestep))
            self.evaluation_scores.append(evaluation_score)

            mean_10_reward = np.mean(self.episode_reward[-10:])
            std_10_reward = np.std(self.episode_reward[-10:])
            mean_100_reward = np.mean(self.episode_reward[-100:])
            std_100_reward = np.std(self.episode_reward[-100:])
            mean_100_eval_score = np.mean(self.evaluation_scores[-100:])
            std_100_eval_score = np.std(self.evaluation_scores[-100:])
            #lst_100_exp_rat = np.array(self.episode_exploration[-100:])/np.array(self.episode_timestep[-100:])
            #mean_100_exp_rat = np.mean(lst_100_exp_rat)
            #std_100_exp_rat = np.std(lst_100_exp_rat)

            wallclock_elapsed = time.time() - training_start
            result[episode-1] = total_step, mean_100_reward, mean_100_eval_score, training_time, wallclock_elapsed
            reached_max_minutes = wallclock_elapsed >= 10 * 60
            reached_max_episodes = episode >= 10000
            reached_goal_mean_reward = mean_100_eval_score >= 475
            #reached_goal_mean_reward=False
            training_is_over = reached_max_minutes or reached_max_episodes or reached_goal_mean_reward

            if training_is_over:
                if reached_max_minutes: print(u'--> reached_max_minutes \u2715')
                if reached_max_episodes: print(u'--> reached_max_episodes \u2715')
                if reached_goal_mean_reward: print(u'--> reached_goal_mean_reward \u2713')
                break

        final_eval_rwd_list = self.evaluateAgent()
        mean_eval_rwd = np.mean(final_eval_rwd_list)
        wallclock_time = time.time() - training_start
        print(f'truncation happens {trunc} times for train {train_count}')
        self.env.close()

        return result, training_time, final_eval_rwd_list, wallclock_time


    def trainNetwork(self):
        T = len(self.rewards)
        discounts = np.logspace(0, T, num=T, base=self.gamma, endpoint=False)
        returns = np.array([np.sum(discounts[:T-t] * self.rewards[t:]) for t in range(T)])

        discounts = torch.FloatTensor(discounts).unsqueeze(1)
        returns = torch.FloatTensor(returns).unsqueeze(1)
        self.logpas = torch.cat(self.logpas)
        policy_loss = -(discounts * returns * self.logpas).mean()
        self.policy_optimizer.zero_grad()
        policy_loss.backward()
        self.policy_optimizer.step()

    def evaluateAgent(self, greedy=True):
        rwd_list = []
        global trunc
        for _ in range(self.MAX_EVAL_EPISODES):
            s, _ = self.env.reset(seed=self.seed)
            rwd_list.append(0)
            begin = 0
            for _ in count():
                if greedy:
                    a = self.policy_model.select_greedy_action(s)
                else:
                    a = self.policy_model.select_action(s)
                s, r, done,truncate, _ = self.env.step(a)
                rwd_list[-1] += r
                if truncate:
                    begin +=1
                    print(f'truncation happening {trunc} times')
                if done or truncate: break    
            
            if (begin):
                print(f'truncation happening {begin} times')
                trunc +=1
                
                

        return rwd_list

In [ ]:
train_count = 0
trunc = 0
reinforce_results = []
best_agent, best_eval_score = None, float('-inf')
seed_list = [420,133,74,317,233]
for seed in seed_list:

    policy_optimizer_fn = lambda net, lr: optim.Adam(net, lr=lr)

    reinforce_instance = REINFORCE(env_id='CartPole-v1', seed=seed, gamma=1.0, optimizerFn = policy_optimizer_fn, optimizerLR = 0.0005,MAX_TRAIN_EPISODES = 500,MAX_EVAL_EPISODES = 1)

    trainRewardsList, trainTimeList, evalRewardsList, wallClockTimeList, final_eval_score = reinforce_instance.runREINFORCE()
    reinforce_results.append(trainRewardsList)
    if final_eval_score > best_eval_score:
        best_eval_score = final_eval_score
        best_agent = reinforce_instance

reinforce_results = np.array(reinforce_results)

In [ ]:
reward_max, steps_max, eval_score_max, train_time_max, wall_clock_max = np.max(reinforce_results, axis=0).T
reward_min, steps_min, eval_score_min, train_time_min, wall_clock_min = np.min(reinforce_results, axis=0).T
reward_avg, steps_avg, eval_score_avg, train_time_avg, wall_clock_avg = np.mean(reinforce_results, axis=0).T
episode_indices = np.arange(len(reward_avg))

plt.style.use('ggplot')  
fig, plot_areas = plt.subplots(5, 1, figsize=(12, 25), sharex='col')
fig.subplots_adjust(hspace=0.5)  
colors = ['blue', 'green', 'red', 'purple', 'orange']
titles = ['Total Steps','Training Reward', 'Evaluation reward',  'Training Duration', 'Wall-clock Time']
y_labels = ['Reward', 'Score', 'Steps', 'Seconds', 'Seconds']
data_max = [reward_max, steps_max, eval_score_max, train_time_max, wall_clock_max]
data_min = [reward_min, steps_min, eval_score_min, train_time_min, wall_clock_min]
data_avg = [reward_avg, steps_avg, eval_score_avg, train_time_avg, wall_clock_avg]

# Generate plots
for ax, title, color, max_data, min_data, avg_data, y_label in zip(plot_areas, titles, colors, data_max, data_min, data_avg, y_labels):
    ax.plot(max_data, linestyle='--', color=color, alpha=0.75)
    ax.plot(min_data, linestyle='--', color=color, alpha=0.75)
    ax.plot(avg_data, label='Average', color=color, linewidth=2)
    ax.fill_between(episode_indices, min_data, max_data, color=color, alpha=0.2)
    ax.set_title(title)
    ax.set_ylabel(y_label)
    ax.legend()

plot_areas[-1].set_xlabel('Episodes')
plt.show()


## Vanilla Policy Gradient (VPG)

Implement the VPG algorithm. We have studied about VPG algorithm in the Lecture. Use the function definitions (given below).

This class implements the VPG Agent, you are required to implement the various methods of this class
as outlined below. Note this class is generic and should work with any permissible Gym environment

```
class VPG():
    def __init__(env, seed, gamma, beta,
                 optimizerFn,
                 optimizerLR,
                 MAX_TRAIN_EPISODES, MAX_EVAL_EPISODES,
                 explorationStrategyTrainFn,
                 explorationStrategyEvalFn)
    def initBookKeeping(self)
    def performBookKeeping(self, train = True)
    def runVPG(self)
    def trainAgent(self)
    def trainPolicyNetwork(self, experiences)
    def evaluateAgent(self)
```

In [ ]:
class VPG():
    def __init__(self, env_id, seed, gamma,
                 policy_model_max_grad_norm,
                 policy_optimizer_fn,
                 policy_optimizer_lr,
                 value_model_max_grad_norm,
                 value_optimizer_fn,
                 value_optimizer_lr,
                 MAX_TRAIN_EPISODES, MAX_EVAL_EPISODES,
                 entropy_loss_weight):
        pass

    def initBookKeeping(self):
        pass

    def performBookKeeping(self, train=True):
        pass

    def runVPG(self):
        pass

    def trainAgent(self):
        pass

    def trainPolicyNetwork(self):
        pass

    def evaluateAgent(self):
        pass


In [ ]:
class VPG():
    def __init__(self, env_id, seed, gamma,
                 policy_model_max_grad_norm,
                 policy_optimizer_fn,
                 policy_optimizer_lr,
                 value_model_max_grad_norm,
                 value_optimizer_fn,
                 value_optimizer_lr,
                 MAX_TRAIN_EPISODES, MAX_EVAL_EPISODES,
                 entropy_loss_weight):

        # 1. Environment initialization using env_id
        self.env = gym.make(env_id)
        self.seed = seed
        reset_env(self.env, seed=seed)  # Seed the environment
        torch.manual_seed(seed)  # Seed PyTorch
        np.random.seed(seed)  # Seed NumPy
        random.seed(seed)

        # 2. Creating P-network
        inDim = self.env.observation_space.shape[0]
        outDim = self.env.action_space.n
        hDim = [128, 64]  # Hidden layer dimensions
        activation = torch.nn.functional.relu
        self.p_network = createPolicyNetworkREINFORCE(inDim, outDim, hDim, activation)

        # 3. Creating V-network
        hDim = [256, 128]
        self.v_network = createPolicyNetworkVPG(inDim, outDim, hDim, activation)

        # 4. Optimizers initialization
        self.p_optimizer = policy_optimizer_fn(self.p_network.parameters(), policy_optimizer_lr)
        self.v_optimizer = value_optimizer_fn(self.v_network.parameters(), value_optimizer_lr)

        # 5. Other variables
        self.gamma = gamma
        self.policy_model_max_grad_norm = policy_model_max_grad_norm
        self.value_model_max_grad_norm = value_model_max_grad_norm
        self.entropy_loss_weight = entropy_loss_weight
        self.MAX_TRAIN_EPISODES = MAX_TRAIN_EPISODES
        self.MAX_EVAL_EPISODES = MAX_EVAL_EPISODES

    def stepint(self, state, env):
        action, is_exploratory, logpa, entropy = self.policy_model.full_pass(state)
        new_state, reward, is_terminal, is_truncated, info = self.env.step(action)

        self.logpas.append(logpa)
        self.entropies.append(entropy)
        self.rewards.append(reward)
        self.values.append(self.value_model(state))

        self.episode_reward[-1] += reward
        self.episode_timestep[-1] += 1
        self.episode_exploration[-1] += int(is_exploratory)
        return new_state, is_terminal, is_truncated

    def trainAgent(self):

        training_start= time.time()
        torch.manual_seed(self.seed) ; np.random.seed(self.seed) ; random.seed(self.seed)
        self.episode_timestep = []
        self.episode_reward = []
        self.episode_seconds = []
        self.episode_exploration = []
        self.evaluation_scores = []
        global train_count
        global trunc
        trunc = 0
        train_count +=1

        self.policy_model = self.p_network
        self.policy_optimizer = self.p_optimizer
        self.value_model = self.v_network
        self.value_optimizer = self.v_optimizer
        max_episodes = self.MAX_TRAIN_EPISODES

        result = np.empty((max_episodes, 5))
        result[:] = np.nan
        training_time = 0

        for episode in range(1, max_episodes + 1):
            episode_start = time.time()
            print(f'episode {episode} and train is called {train_count} times')
            state, _  = self.env.reset(seed=self.seed)
            self.episode_reward.append(0.0)
            self.episode_timestep.append(0.0)
            self.episode_exploration.append(0.0)

            # collect rollout
            self.logpas, self.entropies, self.rewards, self.values = [], [], [], []
            for step in count():
                state, is_terminal, is_truncated = self.stepint(state, self.env)
                if is_terminal:
                    break

            is_failure = is_terminal and not is_truncated
            next_value = 0 if is_failure else self.value_model(state).detach().item()
            self.rewards.append(next_value)
            self.trainPolicyNetwork()

            # stats
            episode_elapsed = time.time() - episode_start
            self.episode_seconds.append(episode_elapsed)
            training_time += episode_elapsed
            evaluation_score= np.mean(self.evaluateAgent())

            total_step = int(np.sum(self.episode_timestep))
            self.evaluation_scores.append(evaluation_score)

            mean_10_reward = np.mean(self.episode_reward[-10:])
            std_10_reward = np.std(self.episode_reward[-10:])
            mean_100_reward = np.mean(self.episode_reward[-100:])
            std_100_reward = np.std(self.episode_reward[-100:])
            mean_100_eval_score = np.mean(self.evaluation_scores[-100:])
            std_100_eval_score = np.std(self.evaluation_scores[-100:])
            #lst_100_exp_rat = np.array(self.episode_exploration[-100:])/np.array(self.episode_timestep[-100:])
            #mean_100_exp_rat = np.mean(lst_100_exp_rat)
            #std_100_exp_rat = np.std(lst_100_exp_rat)

            wallclock_elapsed = time.time() - training_start
            result[episode-1] = total_step, mean_100_reward, mean_100_eval_score, training_time, wallclock_elapsed
            reached_max_minutes = wallclock_elapsed >= 10 * 60
            reached_max_episodes = episode >= 10000
            reached_goal_mean_reward = mean_100_eval_score >= 475
            #reached_goal_mean_reward=False
            training_is_over = reached_max_minutes or reached_max_episodes or reached_goal_mean_reward

            if training_is_over:
                if reached_max_minutes: print(u'--> reached_max_minutes \u2715')
                if reached_max_episodes: print(u'--> reached_max_episodes \u2715')
                if reached_goal_mean_reward: print(u'--> reached_goal_mean_reward \u2713')
                break

        final_eval_rwd_list = self.evaluateAgent()
        mean_eval_rwd = np.mean(final_eval_rwd_list)
        wallclock_time = time.time() - training_start
        self.env.close()
        print(f'truncation happens {trunc} times for train {train_count}')

        return result, training_time, final_eval_rwd_list, wallclock_time

    def trainPolicyNetwork(self):
        T = len(self.rewards)
        discounts = np.logspace(0, T, num=T, base=self.gamma, endpoint=False)
        returns = np.array([np.sum(discounts[:T-t] * self.rewards[t:]) for t in range(T)])
        discounts = torch.FloatTensor(discounts[:-1]).unsqueeze(1)
        returns = torch.FloatTensor(returns[:-1]).unsqueeze(1)

        self.logpas = torch.cat(self.logpas)
        self.entropies = torch.cat(self.entropies)
        self.values = torch.cat(self.values)

        value_error = returns - self.values
        policy_loss = -(discounts * value_error.detach() * self.logpas).mean()
        entropy_loss = -self.entropies.mean()
        loss = policy_loss + self.entropy_loss_weight * entropy_loss
        self.policy_optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(self.policy_model.parameters(),
                                       self.policy_model_max_grad_norm)
        self.policy_optimizer.step()

        value_loss = value_error.pow(2).mul(0.5).mean()
        self.value_optimizer.zero_grad()
        value_loss.backward()
        torch.nn.utils.clip_grad_norm_(self.value_model.parameters(),
                                       self.value_model_max_grad_norm)
        self.value_optimizer.step()

    def evaluateAgent(self, greedy=True):
        rwd_list = []
        global trunc
        for _ in range(self.MAX_EVAL_EPISODES):
            s, _ = self.env.reset(seed=self.seed)
            rwd_list.append(0)
            begin = 0
            for _ in count():
                if greedy:
                    a = self.policy_model.select_greedy_action(s)
                else:
                    a = self.policy_model.select_action(s)
                s, r, done,truncate, _ = self.env.step(a)
                rwd_list[-1] += r
                if truncate:
                  begin +=1
                if done or truncate: break

            if (begin):
                print(f'truncation happening {begin} times')
                trunc +=1
        return rwd_list

    def runVPG(self):
        resultList, trainTimeList, evalRewardsList, wallClockTimeList = self.trainAgent()
        resultEval = self.evaluateAgent()
        finalEvalReward = np.mean(resultEval)

        return resultList, trainTimeList, evalRewardsList, wallClockTimeList, finalEvalReward

In [ ]:
train_count = 0
trunc = 0
vpg_results = []
best_agent, best_eval_score = None, float('-inf')
seed_list = [420,133,74,317,233]
for seed in seed_list:

    policy_model_max_grad_norm = 1
    policy_optimizer_exp = lambda net, lr: optim.Adam(net, lr=lr)
    policy_optimizer_lr = 0.0005

    value_model_max_grad_norm = float('inf')
    value_optimizer_exp = lambda net, lr: optim.RMSprop(net, lr=lr)
    value_optimizer_lr = 0.0007

    entropy_loss_weight = 0.001

    vpg_instance = VPG(env_id = 'CartPole-v1',
                seed = seed,
                gamma = 1.0,
                policy_model_max_grad_norm = 1,
                policy_optimizer_fn = policy_optimizer_exp,
                policy_optimizer_lr = 0.000485,
                value_model_max_grad_norm = float('inf'),
                value_optimizer_fn= value_optimizer_exp,
                value_optimizer_lr = 0.00068,
                MAX_TRAIN_EPISODES = 500,
                MAX_EVAL_EPISODES = 3,
                entropy_loss_weight = 0.0011)

    trainRewardsList, trainTimeList, evalRewardsList, wallClockTimeList, final_eval_score = vpg_instance.runVPG()
    vpg_results.append(trainRewardsList)
    if final_eval_score > best_eval_score:
        best_eval_score = final_eval_score
        best_agent = vpg_instance
vpg_results = np.array(vpg_results)



In [ ]:
reward_max, steps_max, eval_score_max, train_time_max, wall_clock_max = np.max(vpg_results, axis=0).T
reward_min, steps_min, eval_score_min, train_time_min, wall_clock_min = np.min(vpg_results, axis=0).T
reward_avg, steps_avg, eval_score_avg, train_time_avg, wall_clock_avg = np.mean(vpg_results, axis=0).T
episode_indices = np.arange(len(reward_avg))

plt.style.use('ggplot')  
fig, plot_areas = plt.subplots(5, 1, figsize=(12, 25), sharex='col')
fig.subplots_adjust(hspace=0.5)  
colors = ['blue', 'green', 'red', 'purple', 'orange']
titles = ['Total Steps','Training Reward', 'Evaluation reward',  'Training Duration', 'Wall-clock Time']
y_labels = ['Reward', 'Score', 'Steps', 'Seconds', 'Seconds']
data_max = [reward_max, steps_max, eval_score_max, train_time_max, wall_clock_max]
data_min = [reward_min, steps_min, eval_score_min, train_time_min, wall_clock_min]
data_avg = [reward_avg, steps_avg, eval_score_avg, train_time_avg, wall_clock_avg]

# Generate plots
for ax, title, color, max_data, min_data, avg_data, y_label in zip(plot_areas, titles, colors, data_max, data_min, data_avg, y_labels):
    ax.plot(max_data, linestyle='--', color=color, alpha=0.75)
    ax.plot(min_data, linestyle='--', color=color, alpha=0.75)
    ax.plot(avg_data, label='VPG', color=color, linewidth=2)
    ax.fill_between(episode_indices, min_data, max_data, color=color, alpha=0.2)
    ax.set_title(title)
    ax.set_ylabel(y_label)
    ax.legend()

plot_areas[-1].set_xlabel('Episodes')
plt.show()


# Experiments and Plots
<a id="experiments"></a>

Run the NFQ, DQN, Double DQN, Dueling Double DQN, Dueling Double Deep Q Network with Prioritized Experience Replay, REINFORCE and VPG agent on CartPole environment and MountainCar enviroment.

Plot the following for each of the environment separately. Note based on different hyper-parameters and stratgies you use, can you have multiple plots for each of the below.

As you are aware from your past experience, single run of the agent over the environment results in plots that have lot of variance and look very noisy. One way to overcome this is to create several different instances of the environment using different seeds and then average out the results across these and plot these. For all the plots below, you this strategy. You need to run 5 different instances of the environment for each agent. As you have seen in the lecture slides, we plot the maximum and minimum values around the mean in the plots, so this gives us the shaded plot with the mean curve in the between. In this assignment, you are required to do the same. Generate plots with envelop between maximum and minimum value (check the plotQuantity() function in the helper functions).

For each of the quantity of interest, plot each of the agent within the same plot using different colors for the envelop. Choose colors such that that there is clear contrast between the plots corresponding to different agents.

1. Plot mean train rewards vs episodes for Cartpole environment.
2. Plot mean train rewards vs episodes for MountatinCar environment.
3. Plot mean evaluation rewards vs episodes
4. Plot mean evaluation rewards vs episodes
5. Plot total steps vs episode for Cartpole environment.
6. Plot total steps vs episode for MountatinCar environment.
7. Plot train time vs episode for Cartpole environment.
8. Plot train time vs episode for MountatinCar environment.
9. Plot wall clock time vs episode for Cartpole environment.
10. Plot wall clock time vs episode for MountatinCar environment.
11. Based on plots for CartPole environment, what are your observations about different agents. Compare different agents.  
12. Based on plots for MountainCar environment, what are your observations about different agents. Compare different agents. Do these observations concur with the ones for CartPole environment?
13. Based on both the environments, can you generalize some of the findings for the value-based agents? If yes what are those findings?

In [ ]:
def runDeepValueBasedAgents():
    # this function will initialize 5 different instances of the env (using different seeds), run all the agents
    # over these different instances. Collects results and generate the plots state above.
    # generate your plots in the cells below
    # write the answers to part 11, 12 and 13 in the cells below the plot-cells.

In [ ]:
def runDeepPolicyBasedAgents():
    # this function will initialize 5 different instances of the env (using different seeds), run all the agents
    # over these different instances. Collects results and generate the plots state above.
    # generate your plots in the cells below
    # write the answers to part 11, 12 and 13 in the cells below the plot-cells.